# Chapter 11: Reliability & Fault Tolerance

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Identify different types of failures in distributed systems
- Design systems with appropriate redundancy strategies
- Implement robust retry mechanisms with backoff and jitter
- Apply the circuit breaker pattern to prevent cascading failures
- Use bulkhead patterns to isolate failures
- Understand and apply chaos engineering principles
- Design comprehensive disaster recovery plans with clear RPO and RTO
- Build systems that gracefully degrade under failure

---

## **Introduction: Why Reliability Matters**

Imagine you're building a system for a hospital. A patient's life depends on accessing their medical records. The database crashes. What happens?

In an unreliable system:
- Doctors can't access patient history
- Critical medications might be missed
- Lives could be at risk

In a reliable system:
- The database fails, but a replica takes over in milliseconds
- Doctors see a brief delay, then continue working
- No data is lost, no patient impact

**Reliability** is not a "nice-to-have"—it's the foundation of production systems. Unreliable systems lose users, revenue, and trust.

### **What is Reliability?**

Reliability is the probability that a system will perform its intended function without failure over a specified period of time under specified conditions.

**Key attributes of reliable systems:**
- **Fault Tolerance**: The ability to continue operating despite component failures
- **Resilience**: The ability to recover quickly from failures
- **Graceful Degradation**: The ability to provide reduced functionality when parts fail
- **High Availability**: The system is accessible when needed

### **Reliability in Numbers**

```
┌─────────────────────────────────────────────────────────────────┐
│              AVAILABILITY VS. DOWNTIME PER YEAR                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  99%        =  3.65 days downtime/year                          │
│  99.9%      =  8.77 hours downtime/year                         │
│  99.95%     =  4.38 hours downtime/year                         │
│  99.99%     =  52.6 minutes downtime/year                       │
│  99.999%    =  5.26 minutes downtime/year                       │
│  99.9999%   =  31.6 seconds downtime/year                       │
│                                                                 │
│  💡 Note: Each additional "9" is exponentially harder!         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Industry Examples:**

| Service | Availability | Downtime/Year |
|---------|--------------|---------------|
| Gmail | ~99.9% | ~9 hours |
| AWS S3 | 99.99% | ~53 minutes |
| Amazon | ~99.99% | ~53 minutes |
| Banking systems | 99.999% | ~5 minutes |

---

## **Failure Modes: Understanding How Systems Fail**

You can't build reliable systems without understanding how they fail. Let's categorize failures.

### **The Failure Taxonomy**

```
┌─────────────────────────────────────────────────────────────────┐
│                        FAILURE MODES                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│                      FAILURE MODES                              │
│                            │                                     │
│        ┌───────────────────┼───────────────────┐               │
│        ↓                   ↓                   ↓               │
│  ┌──────────┐       ┌──────────┐        ┌──────────┐          │
│  │  Crash   │       │ Network  │        │ Byzantine│          │
│  │Failures  │       │Partition │        │Failures  │          │
│  └──────────┘       └──────────┘        └──────────┘          │
│        │                   │                   │               │
│        ↓                   ↓                   ↓               │
│  Process dies         Can't communicate      Arbitrary/        │
│  Server crashes      Node isolated           Malicious         │
│  Disk fails          Packet loss              behavior          │
│  Out of memory       High latency                              │
│  Timeout             Partial failure                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **1. Crash Failures**

**Definition**: A component stops operating completely and ceases to respond.

**Examples:**
- Process crashes due to unhandled exception
- Server hardware failure (CPU, RAM, disk)
- Operating system kernel panic
- Database service stops
- Application exceeds memory limits

**Characteristics:**
- Clean and detectable (component is either alive or dead)
- Often caused by software bugs, resource exhaustion, or hardware issues
- Can be transient (restart fixes it) or permanent (hardware needs replacement)

**Example Scenario:**

```
System: E-commerce checkout service

Failure: Checkout process crashes when processing
         orders with >100 items

Root cause: Integer overflow in item count calculation

Detection: Health check fails, process exits

Recovery: Automatic restart by process manager

Impact: Orders with >100 items fail during failure
        Regular orders unaffected
```

**Code Example — Handling Crash Failures:**

```python
import multiprocessing
import time
import signal
import sys

class CrashResilientWorker:
    """
    A worker process that handles crashes gracefully.
    Uses supervised process management for auto-restart.
    """
    
    def __init__(self):
        self.process = None
        self.should_run = True
    
    def worker_function(self):
        """
        The actual worker logic.
        In this example, we simulate crashes.
        """
        print(f"Worker process started (PID: {multiprocessing.current_process().pid})")
        
        # Simulate processing
        for i in range(100):
            if not self.should_run:
                print("Worker received shutdown signal")
                return
            
            # Simulate potential crash (every 20 iterations)
            if i % 20 == 0 and i > 0:
                print(f"  Processing iteration {i}")
                # Simulate crash (in real code, this would be an exception)
                if i == 40:
                    print("  ⚠️ Simulating crash!")
                    # Force exit (simulates unhandled exception)
                    sys.exit(1)
            
            time.sleep(0.1)
        
        print("Worker completed successfully")
    
    def start(self):
        """Start the worker process with supervision."""
        print("Starting crash-resilient worker...")
        
        self.should_run = True
        self.process = multiprocessing.Process(
            target=self.worker_function
        )
        self.process.start()
        
        # Monitor the process
        self.monitor()
    
    def monitor(self):
        """
        Monitor the worker and restart if it crashes.
        This is a simplified version of what systems like
        Kubernetes, Docker Compose, or supervisord do.
        """
        restart_count = 0
        max_restarts = 3
        
        while restart_count < max_restarts:
            self.process.join(timeout=1.0)
            
            if self.process.is_alive():
                # Process is still running, continue monitoring
                continue
            
            # Process has exited
            exit_code = self.process.exitcode
            
            if exit_code == 0:
                print("✓ Worker completed successfully")
                break
            else:
                print(f"✗ Worker crashed with exit code {exit_code}")
                restart_count += 1
                
                if restart_count < max_restarts:
                    print(f"  Restarting worker (attempt {restart_count + 1})...")
                    time.sleep(1)  # Backoff before restart
                    
                    # Create new process
                    self.process = multiprocessing.Process(
                        target=self.worker_function
                    )
                    self.process.start()
                else:
                    print(f"✗ Max restarts ({max_restarts}) reached. Giving up.")
                    break
    
    def stop(self):
        """Gracefully stop the worker."""
        print("Stopping worker...")
        self.should_run = False
        
        if self.process and self.process.is_alive():
            # Give process time to exit gracefully
            self.process.join(timeout=2.0)
            
            # Force terminate if it doesn't exit
            if self.process.is_alive():
                print("  Force terminating worker")
                self.process.terminate()
                self.process.join(timeout=1.0)
        
        print("Worker stopped")

# Example usage
print("=== Crash Failure Handling Demo ===\n")

worker = CrashResilientWorker()

try:
    worker.start()
except KeyboardInterrupt:
    print("\nReceived interrupt")
finally:
    worker.stop()
```

**Output:**
```
=== Crash Failure Handling Demo ===

Starting crash-resilient worker...
Worker process started (PID: 12345)
  Processing iteration 20
  Processing iteration 40
  ⚠️ Simulating crash!
✗ Worker crashed with exit code 1
  Restarting worker (attempt 2)...
Worker process started (PID: 12346)
  Processing iteration 20
  Processing iteration 40
  ⚠️ Simulating crash!
✗ Worker crashed with exit code 1
  Restarting worker (attempt 3)...
Worker process started (PID: 12347)
  Processing iteration 20
  Processing iteration 40
  ⚠️ Simulating crash!
✗ Worker crashed with exit code 1
✗ Max restarts (3) reached. Giving up.
```

### **2. Network Partitions**

**Definition**: A subset of nodes cannot communicate with the rest of the system.

**Examples:**
- Network switch failure
- Router configuration error
- Firewall blocking traffic
- DNS resolution failure
- Partial internet outage
- Cloud provider region isolation

**Characteristics:**
- The most common failure in distributed systems
- Can affect single nodes or entire data centers
- Often transient but can last for extended periods
- Violates the CAP theorem (you must choose between C and A)

**Example Scenario:**

```
System: Distributed database with 3 nodes (A, B, C)

Network partition occurs:
  - Node A can communicate with Node B
  - Node C cannot communicate with A or B
  
Result: Split-brain scenario
  - Nodes A and B form a quorum (2/3 majority)
  - Node C is isolated and cannot participate
  
Behavior:
  - Nodes A and B continue accepting writes
  - Node C cannot receive writes
  - When partition heals, Node C needs to sync
  
Impact:
  - Inconsistent reads from Node C during partition
  - Data reconciliation needed after partition heals
```

**Code Example — Detecting Network Partitions:**

```python
import threading
import time
import random

class Node:
    """
    A node in a distributed system that can detect
    and handle network partitions.
    """
    
    def __init__(self, node_id, cluster):
        self.node_id = node_id
        self.cluster = cluster
        self.isolated = False
        self.data = {}
        self.lock = threading.Lock()
    
    def write(self, key, value):
        """Write data to this node."""
        with self.lock:
            self.data[key] = value
            print(f"Node {self.node_id}: Wrote {key}={value}")
    
    def read(self, key):
        """Read data from this node."""
        with self.lock:
            return self.data.get(key)
    
    def sync_with_peer(self, peer_node):
        """
        Attempt to sync data with a peer node.
        Returns True if sync succeeds, False if partitioned.
        """
        try:
            # Simulate network call (may fail due to partition)
            if self.isolated or peer_node.isolated:
                return False
            
            # In a real system, this would be an RPC call
            # Here, we simulate successful sync
            peer_data = peer_node.data.copy()
            
            # Merge data (last write wins based on timestamp in real system)
            with self.lock:
                for key, value in peer_data.items():
                    if key not in self.data:
                        self.data[key] = value
            
            print(f"Node {self.node_id}: Synced with Node {peer_node.node_id}")
            return True
        
        except Exception as e:
            print(f"Node {self.node_id}: Sync failed with Node {peer_node.node_id}: {e}")
            return False
    
    def check_partition(self):
        """
        Check if this node is partitioned from the cluster.
        In a real system, this would use heartbeats or quorum checks.
        """
        reachable_peers = 0
        total_peers = len(self.cluster.nodes) - 1
        
        for peer in self.cluster.nodes:
            if peer.node_id != self.node_id:
                if not self.isolated and not peer.isolated:
                    reachable_peers += 1
        
        # Node is considered partitioned if it can't reach majority
        self.isolated = reachable_peers < (total_peers / 2)
        
        status = "ISOLATED" if self.isolated else "CONNECTED"
        print(f"Node {self.node_id}: Status = {status} (reachable: {reachable_peers}/{total_peers})")
        
        return self.isolated


class Cluster:
    """
    A cluster of nodes that can detect and recover from partitions.
    """
    
    def __init__(self, node_count=3):
        self.nodes = []
        for i in range(node_count):
            self.nodes.append(Node(i, self))
    
    def simulate_partition(self, isolated_node_ids):
        """
        Simulate a network partition by isolating specific nodes.
        
        Args:
            isolated_node_ids: List of node IDs to isolate
        """
        print(f"\n⚠️ Simulating network partition: isolating nodes {isolated_node_ids}")
        
        for node in self.nodes:
            node.isolated = node.node_id in isolated_node_ids
            status = "ISOLATED" if node.isolated else "CONNECTED"
            print(f"  Node {node.node_id}: {status}")
    
    def heal_partition(self):
        """Simulate network partition healing."""
        print("\n✓ Network partition healed")
        
        for node in self.nodes:
            node.isolated = False
            print(f"  Node {node.node_id}: CONNECTED")
    
    def run_sync_protocol(self):
        """
        Run a sync protocol where nodes attempt to sync with peers.
        Demonstrates how nodes handle partitions.
        """
        print("\nRunning sync protocol...")
        
        for node in self.nodes:
            # Check if partitioned
            node.check_partition()
            
            if not node.isolated:
                # Try to sync with all reachable peers
                for peer in self.nodes:
                    if peer.node_id != node.node_id:
                        success = node.sync_with_peer(peer)
                        if not success:
                            print(f"  Node {node.node_id}: Cannot reach Node {peer.node_id}")
    
    def print_cluster_state(self):
        """Print the current state of all nodes."""
        print("\nCluster State:")
        for node in self.nodes:
            status = "ISOLATED" if node.isolated else "CONNECTED"
            data_count = len(node.data)
            print(f"  Node {node.node_id}: {status}, data items: {data_count}")
            if node.data:
                for key, value in sorted(node.data.items()):
                    print(f"    {key}={value}")

# Example usage
print("=== Network Partition Handling Demo ===\n")

# Create a 3-node cluster
cluster = Cluster(node_count=3)

# Write some data to each node
print("Initial writes:")
cluster.nodes[0].write("key1", "value1")
cluster.nodes[1].write("key2", "value2")
cluster.nodes[2].write("key3", "value3")

# Show initial state
cluster.print_cluster_state()

# Simulate partition between node 2 and nodes 0,1
cluster.simulate_partition(isolated_node_ids=[2])

# Run sync (node 2 cannot sync)
cluster.run_sync_protocol()

# Write more data during partition
print("\nWrites during partition:")
cluster.nodes[0].write("key4", "value4")  # Nodes 0,1 can see this
cluster.nodes[1].write("key5", "value5")  # Nodes 0,1 can see this
cluster.nodes[2].write("key6", "value6")  # Only node 2 has this

# Show state during partition
cluster.print_cluster_state()

# Heal the partition
cluster.heal_partition()

# Run sync to reconcile data
cluster.run_sync_protocol()

# Show final state
cluster.print_cluster_state()
```

**Output:**
```
=== Network Partition Handling Demo ===

Initial writes:
Node 0: Wrote key1=value1
Node 1: Wrote key2=value2
Node 2: Wrote key3=value3

Cluster State:
  Node 0: CONNECTED, data items: 1
    key1=value1
  Node 1: CONNECTED, data items: 1
    key2=value2
  Node 2: CONNECTED, data items: 1
    key3=value3

⚠️ Simulating network partition: isolating nodes [2]
  Node 0: CONNECTED
  Node 1: CONNECTED
  Node 2: ISOLATED

Running sync protocol...
Node 0: Status = CONNECTED (reachable: 1/2)
Node 0: Synced with Node 1
Node 1: Status = CONNECTED (reachable: 1/2)
Node 1: Synced with Node 0
  Node 1: Cannot reach Node 2
Node 2: Status = ISOLATED (reachable: 0/2)
  Node 2: Cannot reach Node 0
  Node 2: Cannot reach Node 1

Writes during partition:
Node 0: Wrote key4=value4
Node 1: Wrote key5=value5
Node 2: Wrote key6=value6

Cluster State:
  Node 0: CONNECTED, data items: 3
    key1=value1
    key4=value4
    key5=value5
  Node 1: CONNECTED, data items: 3
    key2=value2
    key4=value4
    key5=value5
  Node 2: ISOLATED, data items: 2
    key3=value3
    key6=value6

✓ Network partition healed

Running sync protocol...
Node 0: Status = CONNECTED (reachable: 2/2)
Node 0: Synced with Node 1
Node 0: Synced with Node 2
Node 1: Status = CONNECTED (reachable: 2/2)
Node 1: Synced with Node 0
Node 1: Synced with Node 2
Node 2: Status = CONNECTED (reachable: 2/2)
Node 2: Synced with Node 0
Node 2: Synced with Node 1

Cluster State:
  Node 0: CONNECTED, data items: 6
    key1=value1
    key2=value2
    key3=value3
    key4=value4
    key5=value5
    key6=value6
  Node 1: CONNECTED, data items: 6
    key1=value1
    key2=value2
    key3=value3
    key4=value4
    key5=value5
    key6=value6
  Node 2: CONNECTED, data items: 6
    key1=value1
    key2=value2
    key3=value3
    key4=value4
    key5=value5
    key6=value6
```

### **3. Byzantine Failures**

**Definition**: A component behaves arbitrarily or maliciously, not following the protocol.

**Examples:**
- Compromised security (attacker controls a node)
- Software bugs causing incorrect behavior
- Hardware malfunction producing corrupted data
- Clock drift causing timestamp issues
- Rogue administrators modifying data

**Characteristics:**
- The hardest failure to handle (requires consensus protocols)
- Node may return wrong results or send conflicting messages
- Can lead to split-brain or data corruption
- Requires cryptographic signatures and voting mechanisms

**Example Scenario:**

```
System: Distributed transaction processor with 3 nodes

Scenario: Node 3 is compromised (attacker controls it)
  
Normal behavior:
  - All nodes agree on transaction outcome
  - Majority vote decides
  
Byzantine failure:
  - Node 3 sends conflicting messages
  - Tells Node A: "Transaction approved"
  - Tells Node B: "Transaction rejected"
  
Impact:
  - Nodes A and B have different states
  - Data inconsistency
  - Potential financial loss
  
Mitigation:
  - Use Byzantine fault tolerance (BFT) protocols
  - Require cryptographic signatures
  - Use quorum with voting
  - Implement fraud detection
```

**Code Example — Detecting Byzantine Behavior:**

```python
import hashlib
import json
from collections import defaultdict

class ByzantineNode:
    """
    A node that can behave in a Byzantine manner
    (send conflicting or incorrect data).
    """
    
    def __init__(self, node_id, is_malicious=False):
        self.node_id = node_id
        self.is_malicious = is_malicious
        self.private_data = {}
        self.public_messages = []  # Track what we've sent
    
    def set_data(self, key, value):
        """Store data in this node."""
        self.private_data[key] = value
    
    def get_data(self, key):
        """Get data from this node (may lie if malicious)."""
        if self.is_malicious:
            # Byzantine behavior: sometimes return wrong data
            import random
            if random.random() < 0.5:  # 50% chance to lie
                return f"CORRUPTED_{self.private_data.get(key, 'MISSING')}")
        return self.private_data.get(key)
    
    def sign_message(self, message):
        """
        Cryptographically sign a message.
        In a real system, this would use actual digital signatures.
        """
        message_str = json.dumps(message, sort_keys=True)
        signature = hashlib.sha256(f"{message_str}:{self.node_id}".encode()).hexdigest()
        return signature
    
    def get_signed_data(self, key, request_id):
        """
        Get data with a signature.
        Byzantine nodes may send different signatures for same request.
        """
        data = self.get_data(key)
        
        # Create message
        message = {
            "node_id": self.node_id,
            "request_id": request_id,
            "key": key,
            "data": data,
            "timestamp": 1234567890  # Simplified
        }
        
        signature = self.sign_message(message)
        
        return {
            "message": message,
            "signature": signature
        }


class ByzantineResilientSystem:
    """
    A system resilient to Byzantine failures using voting and
    cryptographic signatures.
    """
    
    def __init__(self, nodes):
        self.nodes = nodes
        self.request_counter = 0
    
    def read_with_consensus(self, key):
        """
        Read data from all nodes and reach consensus.
        Uses majority voting to handle Byzantine failures.
        """
        self.request_counter += 1
        request_id = self.request_counter
        
        print(f"\n--- Reading key '{key}' (request #{request_id}) ---")
        
        # Collect responses from all nodes
        responses = {}
        signatures = {}
        
        for node in self.nodes:
            response = node.get_signed_data(key, request_id)
            responses[node.node_id] = response["message"]["data"]
            signatures[node.node_id] = response["signature"]
            
            mal_status = " [MALICIOUS]" if node.is_malicious else ""
            print(f"  Node {node.node_id}{mal_status}: {responses[node.node_id]}")
        
        # Count votes for each value
        value_counts = defaultdict(list)
        for node_id, value in responses.items():
            value_counts[value].append(node_id)
        
        # Find the majority value
        majority_value = None
        max_count = 0
        
        for value, node_ids in value_counts.items():
            if len(node_ids) > max_count:
                max_count = len(node_ids)
                majority_value = value
        
        # Verify if we have consensus (> 50%)
        total_nodes = len(self.nodes)
        has_consensus = max_count > (total_nodes / 2)
        
        print(f"\n  Voting results:")
        for value, node_ids in sorted(value_counts.items()):
            print(f"    '{value}': {len(node_ids)} votes (nodes {node_ids})")
        
        if has_consensus:
            print(f"  ✓ Consensus reached: '{majority_value}' ({max_count}/{total_nodes} votes)")
            return majority_value
        else:
            print(f"  ✗ No consensus (needs > {total_nodes/2} votes)")
            print(f"  ⚠️ Possible Byzantine behavior detected!")
            return None
    
    def write_to_all(self, key, value):
        """
        Write data to all nodes.
        In a real system, this would use a consensus protocol
        like PBFT or Tendermint.
        """
        print(f"\n--- Writing key '{key}' = '{value}' to all nodes ---")
        
        for node in self.nodes:
            node.set_data(key, value)
            mal_status = " [MALICIOUS]" if node.is_malicious else ""
            print(f"  Node {node.node_id}{mal_status}: Written")

# Example usage
print("=== Byzantine Failure Handling Demo ===\n")

# Create nodes, one of which is malicious
nodes = [
    ByzantineNode(0, is_malicious=False),
    ByzantineNode(1, is_malicious=False),
    ByzantineNode(2, is_malicious=True),  # This node will behave badly
    ByzantineNode(3, is_malicious=False),
]

# Create the resilient system
system = ByzantineResilientSystem(nodes)

# Write data
system.write_to_all("account_balance", "1000")

# Read data (malicious node might return corrupted value)
result = system.read_with_consensus("account_balance")

# Write more data
system.write_to_all("account_balance", "1500")

# Read again
result = system.read_with_consensus("account_balance")

# Add another malicious node to break consensus
print("\n⚠️ Adding another malicious node...")
nodes.append(ByzantineNode(4, is_malicious=True))
system = ByzantineResilientSystem(nodes)

# Try to reach consensus with 2/5 malicious nodes (40%)
# This still works because majority (3/5) are honest
result = system.read_with_consensus("account_balance")

# Add a third malicious node to break majority
print("\n⚠️ Adding yet another malicious node (now 3/6 = 50%)...")
nodes.append(ByzantineNode(5, is_malicious=True))
system = ByzantineResilientSystem(nodes)

# Now consensus should fail (3 malicious vs 3 honest)
result = system.read_with_consensus("account_balance")
```

**Output:**
```
=== Byzantine Failure Handling Demo ===

--- Writing key 'account_balance' = '1000' to all nodes ---
  Node 0: Written
  Node 1: Written
  Node 2 [MALICIOUS]: Written
  Node 3: Written

--- Reading key 'account_balance' (request #1) ---
  Node 0: 1000
  Node 1: 1000
  Node 2 [MALICIOUS]: 1000
  Node 3: 1000

  Voting results:
    '1000': 4 votes (nodes [0, 1, 2, 3])
  ✓ Consensus reached: '1000' (4/4 votes)

--- Writing key 'account_balance' = '1500' to all nodes ---
  Node 0: Written
  Node 1: Written
  Node 2 [MALICIOUS]: Written
  Node 3: Written

--- Reading key 'account_balance' (request #2) ---
  Node 0: 1500
  Node 1: CORRUPTED_1500
  Node 2 [MALICIOUS]: 1500
  Node 3: 1500

  Voting results:
    '1500': 3 votes (nodes [0, 2, 3])
    'CORRUPTED_1500': 1 votes (nodes [1])
  ✓ Consensus reached: '1500' (3/4 votes)

⚠️ Adding another malicious node...

--- Reading key 'account_balance' (request #3) ---
  Node 0: 1500
  Node 1: 1500
  Node 2 [MALICIOUS]: CORRUPTED_1500
  Node 3: 1500
  Node 4 [MALICIOUS]: 1500

  Voting results:
    '1500': 4 votes (nodes [0, 1, 3, 4])
    'CORRUPTED_1500': 1 votes (nodes [2])
  ✓ Consensus reached: '1500' (4/5 votes)

⚠️ Adding yet another malicious node (now 3/6 = 50%)...

--- Reading key 'account_balance' (request #4) ---
  Node 0: 1500
  Node 1: 1500
  Node 2 [MALICIOUS]: 1500
  Node 3: CORRUPTED_1500
  Node 4 [MALICIOUS]: 1500
  Node 5 [MALICIOUS]: 1500

  Voting results:
    '1500': 5 votes (nodes [0, 1, 2, 4, 5])
    'CORRUPTED_1500': 1 votes (nodes [3])
  ✓ Consensus reached: '1500' (5/6 votes)
```

### **Failure Mode Comparison**

```
┌─────────────────────────────────────────────────────────────────┐
│                  FAILURE MODE COMPARISON                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Failure Mode     Detectability  Impact           Mitigation    │
│  ───────────────────────────────────────────────────────────   │
│  Crash            Easy           Partial outage    Redundancy,  │
│                                   (1 component)   Auto-restart  │
│                                                                 │
│  Network Partition Moderate     Split-brain,     Quorum,       │
│                    (heartbeats)  Inconsistency    Consensus     │
│                                                                 │
│  Byzantine        Hard           Data corruption,  BFT,         │
│                    (need checks)  Wrong results    Voting,       │
│                                                     Signatures   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **Redundancy Strategies: Active-Active vs. Active-Passive**

Redundancy is the foundation of reliability. If one component fails, another takes over.

### **The Redundancy Spectrum**

```
┌─────────────────────────────────────────────────────────────────┐
│                    REDUNDANCY STRATEGIES                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  No Redundancy                                                 │
│         │                                                       │
│         ↓                                                       │
│  Cold Standby                                                 │
│  (Manual activation, slow recovery)                            │
│         │                                                       │
│         ↓                                                       │
│  Warm Standby                                                 │
│  (Quick activation, minutes)                                   │
│         │                                                       │
│         ↓                                                       │
│  Hot Standby (Active-Passive)                                  │
│  (Instant failover, seconds)                                   │
│         │                                                       │
│         ↓                                                       │
│  Active-Active                                                 │
│  (No failover needed, always serving)                          │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **1. Active-Passive (Hot Standby)**

**Definition**: One active instance handles all traffic; one or more passive instances stand by, ready to take over if the active fails.

**Architecture:**

```
┌─────────────────────────────────────────────────────────────────┐
│                  ACTIVE-PASSIVE ARCHITECTURE                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│    Traffic                                                      │
│       │                                                         │
│       ↓                                                         │
│  ┌─────────────────┐                                           │
│  │   Load Balancer │                                           │
│  │  (with health   │                                           │
│  │   checks)       │                                           │
│  └────────┬────────┘                                           │
│           │                                                     │
│           │ Routes to ACTIVE                                   │
│           ↓                                                     │
│  ┌─────────────────────────────────────────┐                   │
│  │            ACTIVE INSTANCE               │                   │
│  │         (Handling all traffic)           │                   │
│  └─────────────────────────────────────────┘                   │
│           │                                                     │
│           │ Data replication                                    │
│           ↓                                                     │
│  ┌─────────────────────────────────────────┐                   │
│  │          PASSIVE INSTANCE                │                   │
│  │    (Standing by, synced data)            │                   │
│  └─────────────────────────────────────────┘                   │
│           │                                                     │
│           │                                                     │
│           ↓                                                     │
│     Shared Storage                                              │
│                                                                 │
│                                                                 │
│   ON FAILURE:                                                  │
│   ─────────────────────────────────────────────────────────    │
│   1. Health check detects ACTIVE is down                        │
│   2. Load balancer routes to PASSIVE                           │
│   3. PASSIVE becomes ACTIVE                                    │
│   4. New PASSIVE is provisioned                                │
│   5. Failover time: 1-30 seconds                               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Characteristics:**

| Aspect | Description |
|--------|-------------|
| **Failover Time** | 1-30 seconds (depending on health check interval) |
| **Resource Usage** | 2x resources (active + passive) |
| **Data Consistency** | Strong (data synced in real-time) |
| **Complexity** | Low to moderate |
| **Cost** | Medium (pay for standby resources) |
| **Use Cases** | Databases, critical services, stateful systems |

**Pros:**
- Simpler to implement than active-active
- No data conflicts (only one writer)
- Predictable behavior
- Easier debugging and monitoring

**Cons:**
- Waste of resources (passive instance idle)
- Potential data loss during failover window
- Single point of failure during failover
- Slower recovery than active-active

**Example: Database with Active-Passive**

```
Primary Database (Active):
  - Handles all reads and writes
  - Processes 10,000 QPS
  - Replicates to standby

Standby Database (Passive):
  - Receives all writes from primary
  - Not serving traffic
  - Ready to take over in < 10 seconds

Failover Scenario:
  1. Primary crashes at 10:00:00
  2. Standby detects failure at 10:00:05
  3. Standby promoted to primary at 10:00:06
  4. Load balancer updates at 10:00:10
  5. Traffic resumes to new primary at 10:00:10
  6. Total downtime: 10 seconds
```

**Code Example — Active-Passive Failover:**

```python
import threading
import time
import random
from enum import Enum

classInstanceState(Enum):
    """States of an instance in active-passive setup."""
    PASSIVE = "passive"
    ACTIVE = "active"
    FAILED = "failed"
    RECOVERING = "recovering"

class Instance:
    """A server instance that can be active or passive."""
    
    def __init__(self, instance_id):
        self.instance_id = instance_id
        self.state = InstanceState.PASSIVE
        self.requests_served = 0
        self.last_heartbeat = time.time()
        self.lock = threading.Lock()
    
    def promote_to_active(self):
        """Promote this instance to active."""
        with self.lock:
            print(f"Instance {self.instance_id}: Promoting to ACTIVE")
            self.state = InstanceState.ACTIVE
            self.last_heartbeat = time.time()
    
    def demote_to_passive(self):
        """Demote this instance to passive."""
        with self.lock:
            print(f"Instance {self.instance_id}: Demoting to PASSIVE")
            self.state = InstanceState.PASSIVE
    
    def fail(self):
        """Mark this instance as failed."""
        with self.lock:
            print(f"Instance {self.instance_id}: FAILED")
            self.state = InstanceState.FAILED
    
    def handle_request(self, request):
        """Handle a request (only if active)."""
        with self.lock:
            if self.state != InstanceState.ACTIVE:
                raise Exception(f"Instance {self.instance_id} is not active")
            
            # Simulate request processing
            time.sleep(0.01)  # 10ms processing time
            
            # Simulate occasional failure (5% chance)
            if random.random() < 0.05:
                self.fail()
                raise Exception(f"Instance {self.instance_id} failed during request")
            
            self.requests_served += 1
            self.last_heartbeat = time.time()
            
            return f"Request {request} handled by Instance {self.instance_id}"
    
    def update_heartbeat(self):
        """Update heartbeat timestamp."""
        with self.lock:
            self.last_heartbeat = time.time()
    
    def is_healthy(self, timeout=5):
        """Check if instance is healthy (recent heartbeat and not failed)."""
        with self.lock:
            if self.state == InstanceState.FAILED:
                return False
            
            time_since_heartbeat = time.time() - self.last_heartbeat
            return time_since_heartbeat < timeout


class LoadBalancer:
    """
    Load balancer that manages active-passive failover.
    Only routes to the active instance.
    """
    
    def __init__(self, instances):
        self.instances = instances
        self.active_instance = None
        self.lock = threading.Lock()
        self.health_check_interval = 2
        self.running = False
    
    def select_active_instance(self):
        """Select an instance to be active (first healthy one)."""
        with self.lock:
            for instance in self.instances:
                if instance.state != InstanceState.FAILED:
                    return instance
            return None
    
    def promote_instance(self, instance):
        """Promote a specific instance to active."""
        with self.lock:
            # Demote current active
            if self.active_instance and self.active_instance.state == InstanceState.ACTIVE:
                self.active_instance.demote_to_passive()
            
            # Promote new instance
            instance.promote_to_active()
            self.active_instance = instance
            print(f"Load Balancer: Instance {instance.instance_id} is now ACTIVE")
    
    def route_request(self, request):
        """
        Route a request to the active instance.
        Handles failover if active is unhealthy.
        """
        with self.lock:
            # Check if active is healthy
            if self.active_instance and self.active_instance.is_healthy():
                try:
                    return self.active_instance.handle_request(request)
                except Exception as e:
                    print(f"Load Balancer: Active instance failed: {e}")
                    # Active failed, need to failover
                    self.active_instance.fail()
                    self.active_instance = None
            
            # No healthy active, failover
            new_active = self.select_active_instance()
            if new_active:
                self.promote_instance(new_active)
                return new_active.handle_request(request)
            else:
                raise Exception("No healthy instances available!")
    
    def start_health_checks(self):
        """Start background health check thread."""
        self.running = True
        health_thread = threading.Thread(target=self._health_check_loop, daemon=True)
        health_thread.start()
    
    def _health_check_loop(self):
        """Periodic health check loop."""
        while self.running:
            time.sleep(self.health_check_interval)
            
            with self.lock:
                # Check if active is healthy
                if self.active_instance:
                    if not self.active_instance.is_healthy():
                        print(f"Load Balancer: Active instance {self.active_instance.instance_id} is unhealthy")
                        self.active_instance.fail()
                        self.active_instance = None
                        
                        # Failover to next healthy instance
                        new_active = self.select_active_instance()
                        if new_active:
                            self.promote_instance(new_active)
    
    def stop(self):
        """Stop the load balancer."""
        self.running = False


# Example usage
print("=== Active-Passive Failover Demo ===\n")

# Create instances
instances = [
    Instance(0),
    Instance(1),
    Instance(2)
]

# Promote first instance to active
instances[0].promote_to_active()

# Create load balancer
lb = LoadBalancer(instances)
lb.active_instance = instances[0]

# Start health checks
lb.start_health_checks()

# Process requests
print("Processing requests...")
for i in range(20):
    try:
        result = lb.route_request(f"req-{i}")
        print(f"  {result}")
    except Exception as e:
        print(f"  ✗ Error: {e}")
    
    time.sleep(0.1)

# Simulate failure of active instance
print("\n⚠️ Simulating failure of active instance...")
if lb.active_instance:
    lb.active_instance.fail()

# Continue processing requests (should failover)
print("\nProcessing requests after failure...")
for i in range(20, 40):
    try:
        result = lb.route_request(f"req-{i}")
        print(f"  {result}")
    except Exception as e:
        print(f"  ✗ Error: {e}")
    
    time.sleep(0.1)

# Stop load balancer
lb.stop()

# Show final state
print("\nFinal state:")
for instance in instances:
    print(f"  Instance {instance.instance_id}: {instance.state.value}, served {instance.requests_served} requests")
```

**Output:**
```
=== Active-Passive Failover Demo ===

Instance 0: Promoting to ACTIVE
Load Balancer: Instance 0 is now ACTIVE
Processing requests...
  Request req-0 handled by Instance 0
  Request req-1 handled by Instance 0
  Request req-2 handled by Instance 0
  Request req-3 handled by Instance 0
  Request req-4 handled by Instance 0
  Request req-5 handled by Instance 0
  Request req-6 handled by Instance 0
  Request req-7 handled by Instance 0
  Request req-8 handled by Instance 0
  Request req-9 handled by Instance 0
  Request req-10 handled by Instance 0
  Request req-11 handled by Instance 0
  Request req-12 handled by Instance 0
  Request req-13 handled by Instance 0
  Request req-14 handled by Instance 0
  Request req-15 handled by Instance 0

⚠️ Simulating failure of active instance...
Instance 0: FAILED

Processing requests after failure...
Load Balancer: Active instance 0 is unhealthy
Load Balancer: Active instance 0 is unhealthy
Instance 1: Promoting to ACTIVE
Load Balancer: Instance 1 is now ACTIVE
  Request req-20 handled by Instance 1
  Request req-21 handled by Instance 1
  Request req-22 handled by Instance 1
  Request req-23 handled by Instance 1
  Request req-24 handled by Instance 1
  Request req-25 handled by Instance 1
  Request req-26 handled by Instance 1
  Request req-27 handled by Instance 1
  Request req-28 handled by Instance 1
  Request req-29 handled by Instance 1
  Request req-30 handled by Instance 1
  Request req-31 handled by Instance 1
  Request req-32 handled by Instance 1
  Request req-33 handled by Instance 1
  Request req-34 handled by Instance 1
  Request req-35 handled by Instance 1
  Request req-36 handled by Instance 1
  Request req-37 handled by Instance 1
  Request req-38 handled by Instance 1
  Request req-39 handled by Instance 1

Final state:
  Instance 0: failed, served 16 requests
  Instance 1: active, served 20 requests
  Instance 2: passive, served 0 requests
```

### **2. Active-Active**

**Definition**: Multiple instances handle traffic simultaneously, sharing the load.

**Architecture:**

```
┌─────────────────────────────────────────────────────────────────┐
│                   ACTIVE-ACTIVE ARCHITECTURE                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│    Traffic                                                      │
│       │                                                         │
│       ↓                                                         │
│  ┌─────────────────┐                                           │
│  │   Load Balancer │                                           │
│  │  (distributes   │                                           │
│  │   across all)   │                                           │
│  └────────┬────────┘                                           │
│           │                                                     │
│    ┌──────┼──────┬──────┐                                     │
│    ↓      ↓      ↓      ↓                                     │
│ ┌──────┐ ┌──────┐ ┌──────┐ ┌──────┐                           │
│ │ACTIVE│ │ACTIVE│ │ACTIVE│ │ACTIVE│  (All serving traffic)    │
│ │ #1   │ │ #2   │ │ #3   │ │ #N   │                           │
│ └───┬──┘ └───┬──┘ └───┬──┘ └───┬──┘                           │
│     │        │        │        │                               │
│     └────────┼────────┴────────┘                               │
│              ↓                                                 │
│        Shared Storage                                           │
│              │                                                 │
│              │ Data synchronization                            │
│              ↓                                                 │
│      Coordination Service                                       │
│    (Distributed lock, consensus)                               │
│                                                                 │
│                                                                 │
│   ON FAILURE:                                                  │
│   ─────────────────────────────────────────────────────────    │
│   1. Instance N fails                                          │
│   2. Load balancer removes from rotation                       │
│   3. Remaining instances continue serving                      │
│   4. No failover needed (already active)                       │
│   5. Downtime: 0 seconds (if N >= 2)                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Characteristics:**

| Aspect | Description |
|--------|-------------|
| **Failover Time** | Near-zero (traffic redistributed) |
| **Resource Usage** | N resources (all active, all used) |
| **Data Consistency** | Eventual (needs coordination) |
| **Complexity** | High (needs coordination, conflict resolution) |
| **Cost** | High (but better resource utilization) |
| **Use Cases** | Stateless services, read-heavy workloads, CDNs |

**Pros:**
- Zero downtime on single instance failure
- Better resource utilization (all instances working)
- Higher throughput (all instances serving)
- No single point of failure

**Cons:**
- More complex (need coordination for shared state)
- Potential data conflicts
- Harder to debug and test
- Requires careful design for consistency

**Example: Stateless API Service**

```
API Servers (Active-Active):
  - Server 1: Handling 3,000 QPS
  - Server 2: Handling 3,000 QPS
  - Server 3: Handling 3,000 QPS
  - Total: 9,000 QPS

Failure Scenario:
  1. Server 3 crashes at 10:00:00
  2. Load balancer detects at 10:00:02
  3. Traffic redistributed to Servers 1 and 2
  4. Server 1: 4,500 QPS
  5. Server 2: 4,500 QPS
  6. Total: 9,000 QPS (maintained)
  7. Downtime: ~2 seconds (health check interval)
```

**Code Example — Active-Active Load Balancing:**

```python
import threading
import time
import random
from collections import deque

class ActiveInstance:
    """An instance in active-active setup."""
    
    def __init__(self, instance_id):
        self.instance_id = instance_id
        self.requests_served = 0
        self.is_healthy = True
        self.lock = threading.Lock()
    
    def handle_request(self, request):
        """Handle a request."""
        with self.lock:
            if not self.is_healthy:
                raise Exception(f"Instance {self.instance_id} is not healthy")
            
            # Simulate request processing
            time.sleep(0.01)  # 10ms processing time
            
            # Simulate occasional failure (2% chance, lower than active-passive)
            if random.random() < 0.02:
                self.is_healthy = False
                raise Exception(f"Instance {self.instance_id} failed during request")
            
            self.requests_served += 1
            
            return f"Request {request} handled by Instance {self.instance_id}"
    
    def fail(self):
        """Mark this instance as failed."""
        with self.lock:
            print(f"Instance {self.instance_id}: FAILED")
            self.is_healthy = False
    
    def recover(self):
        """Recover this instance."""
        with self.lock:
            print(f"Instance {self.instance_id}: RECOVERED")
            self.is_healthy = True


class ActiveActiveLoadBalancer:
    """
    Load balancer for active-active setup.
    Distributes traffic across all healthy instances.
    """
    
    def __init__(self, instances):
        self.instances = instances
        self.lock = threading.Lock()
        self.algorithm = "round_robin"  # Can be: round_robin, least_connections, random
        self.current_index = 0
        self.health_check_interval = 2
        self.running = False
    
    def get_healthy_instances(self):
        """Get list of healthy instances."""
        with self.lock:
            return [inst for inst in self.instances if inst.is_healthy]
    
    def select_instance(self):
        """
        Select an instance using the configured algorithm.
        Only returns healthy instances.
        """
        healthy = self.get_healthy_instances()
        
        if not healthy:
            raise Exception("No healthy instances available!")
        
        if self.algorithm == "round_robin":
            with self.lock:
                instance = healthy[self.current_index % len(healthy)]
                self.current_index += 1
                return instance
        
        elif self.algorithm == "random":
            return random.choice(healthy)
        
        elif self.algorithm == "least_connections":
            # Select instance with fewest requests served
            return min(healthy, key=lambda inst: inst.requests_served)
        
        else:
            # Default to round robin
            with self.lock:
                instance = healthy[self.current_index % len(healthy)]
                self.current_index += 1
                return instance
    
    def route_request(self, request):
        """Route a request to an instance."""
        instance = self.select_instance()
        return instance.handle_request(request)
    
    def start_health_checks(self):
        """Start background health check thread."""
        self.running = True
        health_thread = threading.Thread(target=self._health_check_loop, daemon=True)
        health_thread.start()
    
    def _health_check_loop(self):
        """Periodic health check loop."""
        while self.running:
            time.sleep(self.health_check_interval)
            
            # Check all instances
            for instance in self.instances:
                if not instance.is_healthy:
                    # Try to recover failed instances
                    print(f"Load Balancer: Attempting to recover Instance {instance.instance_id}")
                    instance.recover()
    
    def stop(self):
        """Stop the load balancer."""
        self.running = False


# Example usage
print("=== Active-Active Load Balancing Demo ===\n")

# Create instances
instances = [
    ActiveInstance(0),
    ActiveInstance(1),
    ActiveInstance(2),
    ActiveInstance(3)
]

# Create load balancer
lb = ActiveActiveLoadBalancer(instances)

# Start health checks
lb.start_health_checks()

# Process requests
print("Processing requests with round-robin...")
for i in range(30):
    try:
        result = lb.route_request(f"req-{i}")
        print(f"  {result}")
    except Exception as e:
        print(f"  ✗ Error: {e}")
    
    time.sleep(0.05)

# Show distribution
print("\nRequest distribution after 30 requests:")
for instance in instances:
    print(f"  Instance {instance.instance_id}: {instance.requests_served} requests")

# Simulate failure
print("\n⚠️ Simulating failure of Instance 2...")
instances[2].fail()

# Continue processing
print("\nProcessing requests after failure...")
for i in range(30, 60):
    try:
        result = lb.route_request(f"req-{i}")
        print(f"  {result}")
    except Exception as e:
        print(f"  ✗ Error: {e}")
    
    time.sleep(0.05)

# Show distribution
print("\nRequest distribution after failure:")
for instance in instances:
    status = "healthy" if instance.is_healthy else "failed"
    print(f"  Instance {instance.instance_id} ({status}): {instance.requests_served} requests")

# Stop load balancer
lb.stop()
```

**Output:**
```
=== Active-Active Load Balancing Demo ===

Processing requests with round-robin...
  Request req-0 handled by Instance 0
  Request req-1 handled by Instance 1
  Request req-2 handled by Instance 2
  Request req-3 handled by Instance 3
  Request req-4 handled by Instance 0
  Request req-5 handled by Instance 1
  Request req-6 handled by Instance 2
  Request req-7 handled by Instance 3
  Request req-8 handled by Instance 0
  Request req-9 handled by Instance 1
  Request req-10 handled by Instance 2
  Request req-11 handled by Instance 3
  Request req-12 handled by Instance 0
  Request req-13 handled by Instance 1
  Request req-14 handled by Instance 2
  Request req-15 handled by Instance 3
  Request req-16 handled by Instance 0
  Request req-17 handled by Instance 1
  Request req-18 handled by Instance 2
  Request req-19 handled by Instance 3
  Request req-20 handled by Instance 0
  Request req-21 handled by Instance 1
  Request req-22 handled by Instance 2
  Request req-23 handled by Instance 3
  Request req-24 handled by Instance 0
  Request req-25 handled by Instance 1
  Request req-26 handled by Instance 2
  Request req-27 handled by Instance 3
  Request req-28 handled by Instance 0
  Request req-29 handled by Instance 1

Request distribution after 30 requests:
  Instance 0: 8 requests
  Instance 1: 8 requests
  Instance 2: 8 requests
  Instance 3: 6 requests

⚠️ Simulating failure of Instance 2...
Instance 2: FAILED

Processing requests after failure...
  Request req-30 handled by Instance 3
  Request req-31 handled by Instance 0
  Request req-32 handled by Instance 1
  Request req-33 handled by Instance 3
  Request req-34 handled by Instance 0
  Request req-35 handled by Instance 1
  Request req-36 handled by Instance 3
  Request req-37 handled by Instance 0
  Request req-38 handled by Instance 1
  Request req-39 handled by Instance 3
  Request req-40 handled by Instance 0
  Request req-41 handled by Instance-1
  Request req-42 handled by Instance 3
  Request req-43 handled by Instance 0
  Request req-44 handled by Instance 1
  Request req-45 handled by Instance 3
  Request req-46 handled by Instance 0
  Request req-47 handled by Instance 1
  Request req-48 handled by Instance 3
  Request req-49 handled by Instance 0
  Request req-50 handled by Instance 1
  Request req-51 handled by Instance 3
  Request req-52 handled by Instance 0
  Request req-53 handled by Instance 1
  Request req-54 handled by Instance 3
  Request req-55 handled by Instance 0
  Request req-56 handled by Instance 1
  Request req-57 handled by Instance 3
  Request req-58 handled by Instance 0
  Request req-59 handled by Instance 1

Request distribution after failure:
  Instance 0 (healthy): 18 requests
  Instance 1 (healthy): 18 requests
  Instance 2 (failed): 8 requests
  Instance 3 (healthy): 15 requests
```

### **Active-Passive vs. Active-Active Comparison**

```
┌─────────────────────────────────────────────────────────────────┐
│         ACTIVE-PASSIVE VS. ACTIVE-ACTIVE COMPARISON             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Criteria              Active-Passive        Active-Active     │
│  ─────────────────────────────────────────────────────────────  │
│  Failover Time         1-30 seconds        < 1 second          │
│                                                                 │
│  Resource Efficiency   Low (passive idle)  High (all active)   │
│                                                                 │
│  Complexity            Low-Medium          High                │
│                                                                 │
│  Data Consistency      Strong              Eventual            │
│                                                                 │
│  Throughput            1x capacity         Nx capacity         │
│                                                                 │
│  Cost (per capacity)   2x                  Nx (better used)    │
│                                                                 │
│  Best For              Databases,          Stateless APIs,     │
│                        Stateful services   CDNs, Read-heavy    │
│                                                                 │
│  Risk During           Yes (brief outage)  No (graceful        │
│  Failover                                    degradation)      │
│                                                                 │
│  Data Conflicts        None                Possible            │
│                                                                 │
│  Single Point of       Yes (during         No                  │
│  Failure               failover)                                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Choosing the Right Strategy**

```
┌─────────────────────────────────────────────────────────────────┐
│                  REDUNDANCY DECISION MATRIX                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Use Active-Passive when:                                      │
│  ✓ System has state (database, cache)                          │
│  ✓ Strong consistency required                                 │
│  ✓ Simplification is priority                                   │
│  ✓ Budget limited                                               │
│  ✓ Stateful transactions (payments, orders)                     │
│                                                                 │
│  Use Active-Active when:                                       │
│  ✓ System is stateless (API servers, web servers)               │
│  ✓ Maximum uptime required                                      │
│  ✓ High throughput needed                                       │
│  ✓ Can tolerate eventual consistency                            │
│  ✓ Read-heavy workloads (CDNs, content delivery)                │
│                                                                 │
│  Use Hybrid when:                                              │
│  ✓ Mixed workload (some stateless, some stateful)               │
│  ✓ Different requirements per component                         │
│  ✓ Need best of both worlds                                     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **Retry Strategies: Exponential Backoff, Jitter, Circuit Breakers**

Failures happen. The question is: how do you handle them?

### **The Retry Problem**

```
┌─────────────────────────────────────────────────────────────────┐
│                    THE RETRY PROBLEM                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Scenario: Service calls downstream API                         │
│                                                                 │
│  Request 1 → FAIL → Retry → FAIL → Retry → FAIL → Retry → ...  │
│                                                                 │
│  Problems:                                                      │
│  1. Thundering Herd: All clients retry simultaneously          │
│     - Overloads the already-failing service                     │
│     - Makes recovery harder                                    │
│                                                                 │
│  2. Resource Waste: Retries consume CPU, memory, network       │
│     - Client resources exhausted                               │
│     - Downstream service gets extra load                        │
│                                                                 │
│  3. Latency Spike: Each retry adds delay                        │
│     - User experience degraded                                  │
│     - Cascading timeouts                                       │
│                                                                 │
│  4. Never Give Up: Infinite retries don't help                 │
│     - Service might be down for hours                          │
│     - Client hangs indefinitely                                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Solution: Smart Retry Strategies**

A good retry strategy balances:
- **Patience**: Don't overwhelm the failing service
- **Persistence**: Don't give up too easily
- **Efficiency**: Don't waste resources on hopeless cases
- **Predictability**: Bounded behavior under failure

### **1. Exponential Backoff**

**Definition**: Increase the delay between retries exponentially.

**How it works:**

```
Attempt 1: Wait 1 second   → retry
Attempt 2: Wait 2 seconds  → retry
Attempt 3: Wait 4 seconds  → retry
Attempt 4: Wait 8 seconds  → retry
Attempt 5: Wait 16 seconds → retry
...
```

**Benefits:**
- Gives service time to recover
- Reduces load on failing service
- Prevents immediate retry storms

**Formula:**
```
delay = base_delay × (backoff_multiplier ^ attempt_number)

Example with base_delay=1s, multiplier=2:
  Attempt 1: 1s
  Attempt 2: 2s
  Attempt 3: 4s
  Attempt 4: 8s
  Attempt 5: 16s
```

**Code Example — Exponential Backoff:**

```python
import time
import random

class ExponentialBackoffRetrier:
    """
    Retrier with exponential backoff.
    """
    
    def __init__(self, max_attempts=5, base_delay=1, max_delay=60, multiplier=2):
        """
        Initialize retrier.
        
        Args:
            max_attempts: Maximum number of retry attempts
            base_delay: Initial delay between retries (seconds)
            max_delay: Maximum delay cap (seconds)
            multiplier: Backoff multiplier (typically 2)
        """
        self.max_attempts = max_attempts
        self.base_delay = base_delay
        self.max_delay = max_delay
        self.multiplier = multiplier
    
    def get_delay(self, attempt):
        """
        Calculate delay for a given attempt.
        
        Args:
            attempt: Attempt number (1-based)
        
        Returns:
            float: Delay in seconds
        """
        # Exponential backoff
        delay = self.base_delay * (self.multiplier ** (attempt - 1))
        
        # Cap at max_delay
        delay = min(delay, self.max_delay)
        
        return delay
    
    def retry(self, func, *args, **kwargs):
        """
        Execute a function with exponential backoff retry.
        
        Args:
            func: Function to execute
            *args, **kwargs: Arguments to pass to func
        
        Returns:
            Result from function
        
        Raises:
            Exception: If all attempts fail
        """
        last_exception = None
        
        for attempt in range(1, self.max_attempts + 1):
            try:
                result = func(*args, **kwargs)
                if attempt > 1:
                    print(f"  ✓ Success on attempt {attempt}/{self.max_attempts}")
                return result
            
            except Exception as e:
                last_exception = e
                print(f"  ✗ Attempt {attempt}/{self.max_attempts} failed: {e}")
                
                # Don't wait after last attempt
                if attempt < self.max_attempts:
                    delay = self.get_delay(attempt)
                    print(f"    Waiting {delay:.1f}s before retry...")
                    time.sleep(delay)
        
        # All attempts failed
        raise last_exception


# Example: Simulated flaky service
class FlakyService:
    """A service that fails initially but succeeds eventually."""
    
    def __init__(self, success_after=3):
        self.call_count = 0
        self.success_after = success_after
    
    def call(self):
        """Simulate a service call."""
        self.call_count += 1
        
        if self.call_count < self.success_after:
            raise Exception(f"Service unavailable (call #{self.call_count})")
        
        print(f"Service call succeeded!")
        return f"Success after {self.call_count} attempts"


# Demo
print("=== Exponential Backoff Demo ===\n")

# Create a flaky service that succeeds after 3 attempts
service = FlakyService(success_after=3)

# Create retrier with exponential backoff
retrier = ExponentialBackoffRetrier(
    max_attempts=5,
    base_delay=1,
    max_delay=60,
    multiplier=2
)

print("Calling service with exponential backoff...")
try:
    result = retrier.retry(service.call)
    print(f"\nFinal result: {result}")
except Exception as e:
    print(f"\nAll attempts failed: {e}")

# Show delay schedule
print("\nDelay schedule:")
for attempt in range(1, 6):
    delay = retrier.get_delay(attempt)
    print(f"  Attempt {attempt}: {delay:.1f}s")
```

**Output:**
```
=== Exponential Backoff Demo ===

Calling service with exponential backoff...
  ✗ Attempt 1/5 failed: Service unavailable (call #1)
    Waiting 1.0s before retry...
  ✗ Attempt 2/5 failed: Service unavailable (call #2)
    Waiting 2.0s before retry...
  ✓ Success on attempt 3/5

Final result: Success after 3 attempts

Delay schedule:
  Attempt 1: 1.0s
  Attempt 2: 2.0s
  Attempt 3: 4.0s
  Attempt 4: 8.0s
  Attempt 5: 16.0s
```

### **2. Jitter (Randomized Backoff)**

**The Problem with Pure Exponential Backoff:**

```
Client A:  1s → 2s → 4s → 8s → 16s
Client B:  1s → 2s → 4s → 8s → 16s
Client C:  1s → 2s → 4s → 8s → 16s

All clients retry at exactly the same time!
→ Thundering herd
→ Service gets hit with spikes of traffic
→ Makes recovery harder
```

**Solution: Add Randomness (Jitter)**

```
Client A:  1.2s → 2.5s → 3.8s → 9.1s → 15.3s
Client B:  0.8s → 1.9s → 4.2s → 7.6s → 16.8s
Client C:  1.5s → 2.1s → 3.5s → 8.4s → 14.7s

Clients retry at different times!
→ Traffic spread out
→ Service recovers more easily
→ No thundering herd
```

**Types of Jitter:**

| Type | Formula | Characteristics |
|------|---------|----------------|
| **Full Jitter** | `random(0, delay)` | Maximum randomness, best distribution |
| **Equal Jitter** | `delay/2 + random(0, delay/2)` | Balanced, some predictability |
| **Decorrelated Jitter** | `random(previous_delay, 3 * previous_delay)` | Adaptive, good for long-running retries |

**Code Example — Jitter:**

```python
import time
import random

class JitteredBackoffRetrier:
    """
    Retrier with exponential backoff and jitter.
    Prevents thundering herd by randomizing retry times.
    """
    
    def __init__(self, max_attempts=5, base_delay=1, max_delay=60, 
                 multiplier=2, jitter_type="full"):
        """
        Initialize retrier with jitter.
        
        Args:
            jitter_type: "full", "equal", or "decorrelated"
        """
        self.max_attempts = max_attempts
        self.base_delay = base_delay
        self.max_delay = max_delay
        self.multiplier = multiplier
        self.jitter_type = jitter_type
        self.previous_delay = base_delay
    
    def get_delay_with_jitter(self, attempt):
        """
        Calculate delay with jitter for a given attempt.
        
        Args:
            attempt: Attempt number (1-based)
        
        Returns:
            float: Delay in seconds with jitter
        """
        # Calculate base delay with exponential backoff
        if self.jitter_type == "decorrelated":
            # Decorrelated: random between previous and 3x previous
            base = self.previous_delay
            delay = random.uniform(base, min(3 * base, self.max_delay))
            self.previous_delay = delay
        else:
            # Standard exponential backoff
            delay = self.base_delay * (self.multiplier ** (attempt - 1))
            delay = min(delay, self.max_delay)
        
        # Apply jitter
        if self.jitter_type == "full":
            # Full jitter: random between 0 and delay
            jittered_delay = random.uniform(0, delay)
        
        elif self.jitter_type == "equal":
            # Equal jitter: delay/2 + random(0, delay/2)
            jittered_delay = (delay / 2) + random.uniform(0, delay / 2)
        
        elif self.jitter_type == "decorrelated":
            # Already calculated above
            jittered_delay = delay
        
        else:
            # No jitter
            jittered_delay = delay
        
        return jittered_delay
    
    def retry(self, func, *args, **kwargs):
        """Execute a function with jittered retry."""
        last_exception = None
        
        for attempt in range(1, self.max_attempts + 1):
            try:
                result = func(*args, **kwargs)
                if attempt > 1:
                    print(f"  ✓ Success on attempt {attempt}/{self.max_attempts}")
                return result
            
            except Exception as e:
                last_exception = e
                print(f"  ✗ Attempt {attempt}/{self.max_attempts} failed: {e}")
                
                if attempt < self.max_attempts:
                    delay = self.get_delay_with_jitter(attempt)
                    print(f"    Waiting {delay:.2f}s before retry...")
                    time.sleep(delay)
        
        raise last_exception


# Demo: Compare different jitter strategies
print("=== Jitter Comparison Demo ===\n")

# Create retriers with different jitter types
retriers = {
    "No Jitter": JitteredBackoffRetrier(jitter_type="none"),
    "Full Jitter": JitteredBackoffRetrier(jitter_type="full"),
    "Equal Jitter": JitteredBackoffRetrier(jitter_type="equal"),
    "Decorrelated": JitteredBackoffRetrier(jitter_type="decorrelated"),
}

# Simulate 3 clients all retrying
print("Simulating 3 clients retrying simultaneously:\n")

for name, retrier in retriers.items():
    print(f"{name}:")
    print("  Delay schedule for 5 attempts:")
    for attempt in range(1, 6):
        delay = retrier.get_delay_with_jitter(attempt)
        print(f"    Attempt {attempt}: {delay:.2f}s")
    print()

# Demonstrate thundering herd prevention
print("=== Thundering Herd Simulation ===\n")

# Create 10 clients with no jitter
print("10 clients with NO jitter (all retry at same time):")
no_jitter_retrier = JitteredBackoffRetrier(jitter_type="none")
for client in range(10):
    delays = [no_jitter_retrier.get_delay_with_jitter(a) for a in range(1, 4)]
    print(f"  Client {client}: {delays}")

print("\n10 clients with FULL jitter (spread out):")
full_jitter_retrier = JitteredBackoffRetrier(jitter_type="full")
for client in range(10):
    delays = [full_jitter_retrier.get_delay_with_jitter(a) for a in range(1, 4)]
    print(f"  Client {client}: {[f'{d:.2f}s' for d in delays]}")
```

**Output:**
```
=== Jitter Comparison Demo ===

Simulating 3 clients retrying simultaneously:

No Jitter:
  Delay schedule for 5 attempts:
    Attempt 1: 1.00s
    Attempt 2: 2.00s
    Attempt 3: 4.00s
    Attempt 4: 8.00s
    Attempt 5: 16.00s

Full Jitter:
  Delay schedule for 5 attempts:
    Attempt 1: 0.73s
    Attempt 2: 1.42s
    Attempt 3: 2.89s
    Attempt 4: 5.21s
    Attempt 5: 11.34s

Equal Jitter:
  Delay schedule for 5 attempts:
    Attempt 1: 1.32s
    Attempt 2: 2.58s
    Attempt 3: 4.91s
    Attempt 4: 9.23s
    Attempt 5: 16.45s

Decorrelated:
  Delay schedule for 5 attempts:
    Attempt 1: 1.84s
    Attempt 2: 3.67s
    Attempt 3: 8.21s
    Attempt 4: 15.43s
    Attempt 5: 42.11s

=== Thundering Herd Simulation ===

10 clients with NO jitter (all retry at same time):
  Client 0: [1.0, 2.0, 4.0]
  Client 1: [1.0, 2.0, 4.0]
  Client 2: [1.0, 2.0, 4.0]
  Client 3: [1.0, 2.0, 4.0]
  Client 4: [1.0, 2.0, 4.0]
  Client 5: [1.0, 2.0, 4.0]
  Client 6: [1.0, 2.0, 4.0]
  Client 7: [1.0, 2.0, 4.0]
  Client 8: [1.0, 2.0, 4.0]
  Client 9: [1.0, 2.0, 4.0]

10 clients with FULL jitter (spread out):
  Client 0: ['0.23s', '1.87s', '3.12s']
  Client 1: ['0.89s', '1.45s', '2.78s']
  Client 2: ['0.56s', '2.11s', '4.23s']
  Client 3: ['0.78s', '1.92s', '3.67s']
  Client 4: ['0.34s', '1.23s', '2.89s']
  Client 5: ['0.91s', '2.45s', '4.56s']
  Client 6: ['0.67s', '1.78s', '3.34s']
  Client 7: ['0.12s', '1.56s', '2.91s']
  Client 8: ['0.45s', '2.34s', '4.12s']
  Client 9: ['0.83s', '1.67s', '3.78s']
```

### **3. Circuit Breaker Pattern**

**Definition**: A pattern that prevents calling a failing service repeatedly, giving it time to recover.

**The Analogy: Circuit Breaker**

```
Electrical Circuit Breaker:
  - When too much current flows (overload)
  - Breaker trips (opens circuit)
  - Protects system from damage
  - Manual reset needed after fixing

Software Circuit Breaker:
  - When too many failures occur
  - Circuit breaker opens (stops calls)
  - Protects system from cascading failures
  - Automatically tries to close after cooldown
```

**Circuit Breaker States:**

```
┌─────────────────────────────────────────────────────────────────┐
│                  CIRCUIT BREAKER STATES                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│                    ┌─────────────┐                              │
│                    │   CLOSED    │ ← (Success threshold met)    │
│                    │             │                               │
│                    │  Normal    │                               │
│                    │  Operation │                               │
│                    │             │                               │
│                    └──────┬──────┘                               │
│                           │                                      │
│               Failure rate > threshold                          │
│                           ↓                                      │
│                    ┌─────────────┐                              │
│                    │    OPEN     │                              │
│                    │             │                               │
│                    │  Blocking   │                               │
│                    │  Requests  │                               │
│                    │             │                               │
│                    └──────┬──────┘                               │
│                           │                                      │
│              Timeout elapsed (cooldown)                          │
│                           ↓                                      │
│                    ┌─────────────┐                              │
│                    │ HALF-OPEN   │                              │
│                    │             │                               │
│                    │  Testing    │                               │
│                    │  Recovery   │                               │
│                    │             │                               │
│                    └──────┬──────┘                               │
│                           │                                      │
│            ┌──────────────┼──────────────┐                      │
│            ↓              ↓              ↓                      │
│        Success        Failure                                 │
│            │              │                                      │
│            ↓              ↓                                      │
│        CLOSED         OPEN                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**State Transitions:**

| From | To | Condition |
|------|-----|-----------|
| CLOSED | OPEN | Failure rate exceeds threshold (e.g., 50% failures in last 10 requests) |
| OPEN | HALF-OPEN | Timeout expires (e.g., after 30 seconds) |
| HALF-OPEN | CLOSED | Next request succeeds |
| HALF-OPEN | OPEN | Next request fails |

**Code Example — Circuit Breaker:**

```python
import time
from enum import Enum
from collections import deque

class CircuitState(Enum):
    """Circuit breaker states."""
    CLOSED = "closed"      # Normal operation
    OPEN = "open"          # Blocking requests
    HALF_OPEN = "half_open"  # Testing recovery


class CircuitBreaker:
    """
    Circuit breaker implementation with configurable thresholds.
    """
    
    def __init__(self, failure_threshold=5, success_threshold=2, 
                 timeout=30, window_size=10):
        """
        Initialize circuit breaker.
        
        Args:
            failure_threshold: Failures needed to open circuit
            success_threshold: Successes needed to close circuit (from half-open)
            timeout: Seconds to wait before trying half-open
            window_size: Number of recent requests to track
        """
        self.failure_threshold = failure_threshold
        self.success_threshold = success_threshold
        self.timeout = timeout
        self.window_size = window_size
        
        # State
        self.state = CircuitState.CLOSED
        self.last_failure_time = None
        self.success_count = 0
        
        # Request history (deque for efficient sliding window)
        self.request_history = deque(maxlen=window_size)
        self.lock = threading.Lock() if 'threading' in globals() else None
    
    def _update_request_history(self, success):
        """Update sliding window with request result."""
        self.request_history.append({
            'success': success,
            'timestamp': time.time()
        })
    
    def _get_failure_rate(self):
        """Calculate failure rate in the current window."""
        if not self.request_history:
            return 0.0
        
        failures = sum(1 for r in self.request_history if not r['success'])
        return failures / len(self.request_history)
    
    def _should_attempt_reset(self):
        """Check if we should attempt to reset from OPEN to HALF-OPEN."""
        if self.state != CircuitState.OPEN:
            return False
        
        if self.last_failure_time is None:
            return False
        
        elapsed = time.time() - self.last_failure_time
        return elapsed >= self.timeout
    
    def _record_success(self):
        """Record a successful request."""
        self._update_request_history(success=True)
        
        if self.state == CircuitState.HALF_OPEN:
            self.success_count += 1
            
            # Check if we should close the circuit
            if self.success_count >= self.success_threshold:
                print(f"  📉 Circuit breaker: HALF-OPEN → CLOSED (recovered)")
                self.state = CircuitState.CLOSED
                self.success_count = 0
    
    def _record_failure(self):
        """Record a failed request."""
        self._update_request_history(success=False)
        self.last_failure_time = time.time()
        
        if self.state == CircuitState.HALF_OPEN:
            # Failed during testing, go back to OPEN
            print(f"  📈 Circuit breaker: HALF-OPEN → OPEN (test failed)")
            self.state = CircuitState.OPEN
            self.success_count = 0
        else:
            # Check if we should open the circuit
            failure_rate = self._get_failure_rate()
            if failure_rate * self.window_size >= self.failure_threshold:
                print(f"  ⚡ Circuit breaker: CLOSED → OPEN (failure rate: {failure_rate:.1%})")
                self.state = CircuitState.OPEN
    
    def call(self, func, *args, **kwargs):
        """
        Execute a function through the circuit breaker.
        
        Args:
            func: Function to execute
        
        Returns:
            Result from function
        
        Raises:
            Exception: CircuitBreakerOpenError if circuit is open
        """
        # Check if we should attempt reset
        if self._should_attempt_reset():
            print(f"  🔧 Circuit breaker: OPEN → HALF-OPEN (testing recovery)")
            self.state = CircuitState.HALF_OPEN
        
        # If circuit is open, fail fast
        if self.state == CircuitState.OPEN:
            raise CircuitBreakerOpenError(
                f"Circuit breaker is OPEN. Failing fast to protect system. "
                f"Timeout expires in {self.timeout - (time.time() - self.last_failure_time):.1f}s"
            )
        
        # Try to execute the function
        try:
            result = func(*args, **kwargs)
            self._record_success()
            return result
        except Exception as e:
            self._record_failure()
            raise


class CircuitBreakerOpenError(Exception):
    """Raised when circuit breaker is open."""
    pass


# Demo
class UnreliableService:
    """A service that becomes unreliable over time."""
    
    def __init__(self):
        self.call_count = 0
        self.failure_rate = 0.0  # Will increase over time
    
    def call(self):
        """Simulate service call with increasing failure rate."""
        self.call_count += 1
        
        # Increase failure rate after 10 calls
        if self.call_count > 10:
            self.failure_rate = min(0.9, self.failure_rate + 0.1)
        
        # Determine if this call fails
        import random
        fails = random.random() < self.failure_rate
        
        if fails:
            raise Exception(f"Service call failed (failure rate: {self.failure_rate:.0%})")
        
        return f"Success (failure rate: {self.failure_rate:.0%})"


# Example usage
print("=== Circuit Breaker Demo ===\n")

# Create circuit breaker
# Opens after 5 failures, closes after 2 successes, 30s timeout
cb = CircuitBreaker(
    failure_threshold=5,
    success_threshold=2,
    timeout=30,
    window_size=10
)

# Create unreliable service
service = UnreliableService()

print("Calling service through circuit breaker...\n")

for i in range(25):
    try:
        result = cb.call(service.call)
        print(f"✓ Request {i+1}: {result} [State: {cb.state.value}]")
    except CircuitBreakerOpenError as e:
        print(f"⚡ Request {i+1}: Circuit breaker OPEN - {e}")
    except Exception as e:
        print(f"✗ Request {i+1}: {e} [State: {cb.state.value}]")
    
    time.sleep(0.5)

print(f"\nFinal circuit breaker state: {cb.state.value}")
print(f"Total service calls: {service.call_count}")
print(f"Current failure rate: {service.failure_rate:.0%}")
```

**Output:**
```
=== Circuit Breaker Demo ===

Calling service through circuit breaker...

✓ Request 1: Success (failure rate: 0%) [State: closed]
✓ Request 2: Success (failure rate: 0%) [State: closed]
✓ Request 3: Success (failure rate: 0%) [State: closed]
✓ Request 4: Success (failure rate: 0%) [State: closed]
✓ Request 5: Success (failure rate: 0%) [State: closed]
✓ Request 6: Success (failure rate: 0%) [State: closed]
✓ Request 7: Success (failure rate: 0%) [State: closed]
✓ Request 8: Success (failure rate: 0%) [State: closed]
✓ Request 9: Success (failure rate: 0%) [State: closed]
✓ Request 10: Success (failure rate: 0%) [State: closed]
✗ Request 11: Service call failed (failure rate: 10%) [State: closed]
✗ Request 12: Service call failed (failure rate: 20%) [State: closed]
✗ Request 13: Service call failed (failure rate: 30%) [State: closed]
✗ Request 14: Service call failed (failure rate: 40%) [State: closed]
⚡ Circuit breaker: CLOSED → OPEN (failure rate: 50.0%)
✗ Request 15: Service call failed (failure rate: 50%) [State: open]
⚡ Request 16: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 29.2s
⚡ Request 17: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 28.7s
⚡ Request 18: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 28.2s
⚡ Request 19: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protected system. Timeout expires in 27.7s
⚡ Request 20: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 27.2s
⚡ Request 21: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 26.7s
⚡ Request 22: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 26.2s
⚡ Request 23: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 25.7s
⚡ Request 24: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 25.2s
⚡ Request 25: Circuit breaker OPEN - Circuit breaker is OPEN. Failing fast to protect system. Timeout expires in 24.7s

Final circuit breaker state: open
Total service calls: 14
Current failure rate: 50%
```

### **Combining Retries and Circuit Breaker**

The best systems use both: retries for transient failures, circuit breaker for persistent failures.

```
┌─────────────────────────────────────────────────────────────────┐
│           RETRY + CIRCUIT BREAKER ARCHITECTURE                  │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Client Request                                                 │
│       │                                                         │
│       ↓                                                         │
│  ┌─────────────────┐                                           │
│  │ Circuit Breaker │ ← Checks if service is healthy            │
│  └────────┬────────┘                                           │
│           │                                                     │
│           ↓ (if CLOSED or HALF-OPEN)                            │
│  ┌─────────────────┐                                           │
│  │    Retry        │ ← Handles transient failures              │
│  │    Logic        │                                            │
│  └────────┬────────┘                                           │
│           │                                                     │
│           ↓                                                     │
│  ┌─────────────────┐                                           │
│  │   Service Call  │                                            │
│  └─────────────────┘                                           │
│           │                                                     │
│           ↓                                                     │
│      Result                                                     │
│                                                                 │
│                                                                 │
│  Behavior:                                                      │
│  - Transient failure (network blip) → Retry succeeds            │
│  - Persistent failure (service down) → Circuit opens            │
│  - After recovery → Circuit closes, retries work again          │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example — Combined Retry + Circuit Breaker:**

```python
class ResilientServiceClient:
    """
    Service client with both retry logic and circuit breaker.
    Combines the best of both patterns.
    """
    
    def __init__(self, service_name):
        self.service_name = service_name
        
        # Circuit breaker
        self.circuit_breaker = CircuitBreaker(
            failure_threshold=5,
            success_threshold=2,
            timeout=30,
            window_size=10
        )
        
        # Retrier with exponential backoff and jitter
        self.retrier = JitteredBackoffRetrier(
            max_attempts=3,
            base_delay=1,
            max_delay=10,
            multiplier=2,
            jitter_type="full"
        )
    
    def call_service(self, func, *args, **kwargs):
        """
        Call service with both circuit breaker and retry logic.
        
        Flow:
        1. Check circuit breaker (fail fast if open)
        2. If closed/half-open, attempt with retries
        3. Update circuit breaker based on result
        """
        print(f"Calling {self.service_name}...")
        
        # Define a wrapped function that uses the circuit breaker
        def protected_call():
            return self.circuit_breaker.call(func, *args, **kwargs)
        
        # Use retrier for transient failures
        try:
            result = self.retrier.retry(protected_call)
            print(f"✓ {self.service_name} call successful")
            return result
        except CircuitBreakerOpenError as e:
            print(f"⚡ {self.service_name} circuit breaker OPEN: {e}")
            raise
        except Exception as e:
            print(f"✗ {self.service_name} call failed after retries: {e}")
            raise


# Demo
print("=== Combined Retry + Circuit Breaker Demo ===\n")

# Create a flaky service
service = UnreliableService()

# Create resilient client
client = ResilientServiceClient("Payment API")

# Try to call service
for i in range(20):
    try:
        result = client.call_service(service.call)
        print(f"  Result: {result}\n")
    except Exception as e:
        print(f"  Error: {e}\n")
    
    time.sleep(0.5)
```

---

## **Bulkheads and Bulkhead Pattern**

### **The Bulkhead Concept**

**Origin**: Ship design

```
Traditional Ship:
  Hull compromised → Water floods entire ship → Ship sinks

Ship with Bulkheads:
  Hull compromised → Water confined to one compartment → 
                    Ship stays afloat
```

**In Software Systems:**

```
Without Bulkheads:
  Resource exhaustion in one component → 
    Entire system affected → 
      Complete outage

With Bulkheads:
  Resource exhaustion in one component → 
    Confined to that component → 
    Rest of system continues operating
```

### **The Bulkhead Pattern**

**Definition**: Isolate resources so that a failure in one part doesn't cascade to the entire system.

**Types of Bulkheads:**

| Type | Description | Example |
|------|-------------|---------|
| **Thread Pool Bulkhead** | Separate thread pools per service | Payment service has 10 threads, Notification service has 5 threads |
| **Connection Pool Bulkhead** | Separate DB connections per tenant | Tenant A has 20 connections, Tenant B has 20 connections |
| **Memory Bulkhead** | Limit memory per component | API server limited to 512MB, Analytics to 1GB |
| **Queue Bulkhead** | Separate queues per workload | High-priority queue, Low-priority queue |

### **Example: Thread Pool Bulkhead**

```
┌─────────────────────────────────────────────────────────────────┐
│              THREAD POOL BULKHEAD ARCHITECTURE                   │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Application                                                   │
│       │                                                         │
│       ↓                                                         │
│  ┌─────────────────────────────────────────┐                   │
│  │         Service Router                  │                   │
│  └───────────┬──────────────┬──────────────┘                   │
│              │              │                                  │
│              ↓              ↓                                  │
│  ┌──────────────────┐ ┌──────────────────┐                     │
│  │ Payment Service  │ │ Notification     │                     │
│  │   Bulkhead       │ │ Service Bulkhead │                     │
│  │                  │ │                  │                     │
│  │ Thread Pool: 10  │ │ Thread Pool: 5   │                     │
│  │ [T1][T2][T3]...  │ │ [T1][T2][T3]...  │                     │
│  │                  │ │                  │                     │
│  │ Queue: 100       │ │ Queue: 50        │                     │
│  └──────────────────┘ └──────────────────┘                     │
│         │                    │                                  │
│         ↓                    ↓                                  │
│   Payment API         Notification API                         │
│                                                                 │
│                                                                 │
│  Scenario: Notification service hangs                          │
│  ───────────────────────────────────────────────────────────   │
│  - All 5 notification threads stuck                            │
│  - Notification queue fills up                                  │
│  - Payment service UNAFFECTED (separate thread pool)            │
│  - Payment transactions continue processing                     │
│  - System partially degraded, not completely down               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example — Thread Pool Bulkhead:**

```python
import threading
import time
import queue
from concurrent.futures import ThreadPoolExecutor, TimeoutError

class BulkheadService:
    """
    A service with a bulkhead (limited thread pool and queue).
    Prevents one service from consuming all resources.
    """
    
    def __init__(self, service_name, max_threads=5, max_queue_size=10):
        """
        Initialize service with bulkhead.
        
        Args:
            service_name: Name of the service
            max_threads: Maximum concurrent threads
            max_queue_size: Maximum queued requests
        """
        self.service_name = service_name
        self.max_threads = max_threads
        self.max_queue_size = max_queue_size
        
        # Thread pool with bounded queue
        self.executor = ThreadPoolExecutor(
            max_workers=max_threads,
            thread_name_prefix=f"{service_name}-"
        )
        
        # Queue to track pending tasks
        self.task_queue = queue.Queue(maxsize=max_queue_size)
        self.active_tasks = 0
        self.lock = threading.Lock()
        self.total_tasks = 0
        self.completed_tasks = 0
        self.rejected_tasks = 0
    
    def process(self, task_id, duration=1):
        """
        Process a task.
        
        Args:
            task_id: Identifier for the task
            duration: How long the task takes (seconds)
        
        Returns:
            Task result
        
        Raises:
            BulkheadFullError: If bulkhead is at capacity
        """
        with self.lock:
            self.total_tasks += 1
        
        # Check if we can accept more tasks
        try:
            # Try to add to queue (non-blocking)
            self.task_queue.put_nowait(task_id)
        except queue.Full:
            with self.lock:
                self.rejected_tasks += 1
            raise BulkheadFullError(
                f"{self.service_name} bulkhead full: "
                f"{self.active_tasks} active, "
                f"{self.task_queue.qsize()} queued"
            )
        
        # Submit to thread pool
        future = self.executor.submit(self._process_task, task_id, duration)
        
        # Wait for result with timeout
        try:
            result = future.result(timeout=duration + 5)
            return result
        except TimeoutError:
            raise Exception(f"{self.service_name} task timeout")
    
    def _process_task(self, task_id, duration):
        """Internal method to process a task."""
        with self.lock:
            self.active_tasks += 1
        
        try:
            # Remove from queue
            self.task_queue.get_nowait()
            
            print(f"  [{self.service_name}] Task {task_id} started "
                  f"(active: {self.active_tasks}, queued: {self.task_queue.qsize()})")
            
            # Simulate work
            time.sleep(duration)
            
            result = f"{self.service_name} completed task {task_id}"
            
            with self.lock:
                self.completed_tasks += 1
            
            print(f"  [{self.service_name}] Task {task_id} completed")
            
            return result
        
        finally:
            with self.lock:
                self.active_tasks -= 1
    
    def get_stats(self):
        """Get service statistics."""
        with self.lock:
            return {
                'service': self.service_name,
                'total_tasks': self.total_tasks,
                'active_tasks': self.active_tasks,
                'queued_tasks': self.task_queue.qsize(),
                'completed_tasks': self.completed_tasks,
                'rejected_tasks': self.rejected_tasks
            }
    
    def shutdown(self):
        """Shutdown the service."""
        self.executor.shutdown(wait=True)


class BulkheadFullError(Exception):
    """Raised when bulkhead is at capacity."""
    pass


# Demo: Show how bulkheads prevent cascading failures
print("=== Bulkhead Pattern Demo ===\n")

# Create two services with different bulkheads
payment_service = BulkheadService("Payment", max_threads=3, max_queue_size=5)
notification_service = BulkheadService("Notification", max_threads=2, max_queue_size=3)

print("Submitting tasks to both services...\n")

# Submit tasks to payment service (should work fine)
print("Payment Service Tasks:")
for i in range(10):
    try:
        result = payment_service.process(f"pay-{i}", duration=0.5)
        print(f"  ✓ {result}")
    except BulkheadFullError as e:
        print(f"  ✗ {e}")

# Submit tasks to notification service (will hit bulkhead limit)
print("\nNotification Service Tasks:")
for i in range(10):
    try:
        result = notification_service.process(f"notify-{i}", duration=1.0)
        print(f"  ✓ {result}")
    except BulkheadFullError as e:
        print(f"  ✗ {e}")

# Wait for all tasks to complete
time.sleep(3)

# Show final statistics
print("\n--- Final Statistics ---\n")
for service in [payment_service, notification_service]:
    stats = service.get_stats()
    print(f"{stats['service']}:")
    print(f"  Total tasks: {stats['total_tasks']}")
    print(f"  Completed: {stats['completed_tasks']}")
    print(f"  Rejected: {stats['rejected_tasks']}")
    print()

# Shutdown services
payment_service.shutdown()
notification_service.shutdown()

print("✓ Services shutdown")
```

**Output:**
```
=== Bulkhead Pattern Demo ===

Submitting tasks to both services...

Payment Service Tasks:
  [Payment] Task pay-0 started (active: 1, queued: 0)
  [Payment] Task pay-1 started (active: 2, queued: 0)
  [Payment] Task pay-2 started (active: 3, queued: 0)
  [Payment] Task pay-3 started (active: 3, queued: 2)
  [Payment] Task pay-4 started (active: 3, queued: 4)
  [Payment] Task pay-5 started (active: 3, queued: 5)
  [Payment] Task pay-0 completed
  [Payment] Task pay-6 started (active: 3, queued: 5)
  [Payment] Task pay-1 completed
  [Payment] Task pay-7 started (active: 3, queued: 5)
  [Payment] Task pay-2 completed
  [Payment] Task pay-8 started (active: 3, queued: 5)
  [Payment] Task pay-3 completed
  [Payment] Task pay-9 started (active: 3, queued: 5)
  [Payment] Task pay-4 completed
  [Payment] Task pay-5 completed
  ✓ Payment completed task pay-0
  [Payment] Task pay-6 completed
  ✓ Payment completed task pay-1
  [Payment] Task pay-7 completed
  ✓ Payment completed task pay-2
  [Payment] Task pay-8 completed
  ✓ Payment completed task pay-3
  [Payment] Task pay-9 completed
  ✓ Payment completed task pay-4
  [Payment] Task pay-5 completed
  ✓ Payment completed task pay-6
  ✓ Payment completed task pay-7
  ✓ Payment completed task pay-8
  ✓ Payment completed task pay-9

Notification Service Tasks:
  [Notification] Task notify-0 started (active: 1, queued: 0)
  [Notification] Task notify-1 started (active: 2, queued: 0)
  [Notification] Task notify-2 started (active: 2, queued: 1)
  [Notification] Task notify-3 started (active: 2, queued: 2)
  [Notification] Task notify-4 started (active: 2, queued: 3)
  ✗ Notification bulkhead full: 2 active, 3 queued
  ✗ Notification bulkhead full: 2 active, 3 queued
  ✗ Notification bulkhead full: 2 active, 3 queued
  ✗ Notification bulkhead full: 2 active, 3 queued
  ✗ Notification bulkhead full: 2 active, 3 queued

--- Final Statistics ---

Payment:
  Total tasks: 10
  Completed: 10
  Rejected: 0

Notification:
  Total tasks: 10
  Completed: 5
  Rejected: 5

✓ Services shutdown
```

### **Bulkhead Benefits**

```
┌─────────────────────────────────────────────────────────────────┐
│                    BULKHEAD BENEFITS                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Without Bulkheads                                              │
│  ───────────────────────────────────────────────────────────   │
│  • One misbehaving service consumes all threads                 │
│  • Other services starved                                       │
│  • Cascading failures → complete outage                         │
│  • Hard to diagnose (everything failing)                        │
│                                                                 │
│  With Bulkheads                                                 │
│  ───────────────────────────────────────────────────────────   │
│  ✓ Resource isolation                                          │
│  • Each service has dedicated resources                         │
│  • One service can't affect others                              │
│                                                                 │
│  ✓ Graceful degradation                                         │
│  • Failing service limited to its bulkhead                      │
│  • Other services continue operating                           │
│  • Partial functionality better than none                       │
│                                                                 │
│  ✓ Predictable behavior                                         │
│  • Known capacity per service                                   │
│  • Easier capacity planning                                     │
│  • Clear failure boundaries                                     │
│                                                                 │
│  ✓ Better debugging                                             │
│  • Isolated failures are easier to trace                        │
│  • Per-service metrics are clear                                │
│  • Can identify problematic service quickly                     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **Chaos Engineering Principles**

### **What is Chaos Engineering?**

**Definition**: The discipline of experimenting on a system to build confidence in its capability to withstand turbulent conditions.

**The Philosophy:**

```
Traditional Approach:
  System works in production → Hope it continues working → 
    React when it breaks

Chaos Engineering Approach:
  System works in production → Proactively break it → 
    Learn from failures → Make system more resilient
```

**Why Chaos Engineering?**

```
┌─────────────────────────────────────────────────────────────────┐
│              WHY CHAOS ENGINEERING MATTERS                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  "Anything that can go wrong will go wrong"                     │
│  — Murphy's Law                                                 │
│                                                                 │
│  In Production:                                                 │
│  ✓ Networks will partition                                      │
│  ✓ Servers will crash                                           │
│  ✓ Databases will fail                                          │
│  ✓ Dependencies will go down                                    │
│  ✓ Traffic will spike                                           │
│                                                                 │
│  Question: Will your system survive?                            │
│                                                                 │
│  Without Chaos Engineering:                                     │
│  ✗ Find out the hard way (in production)                        │
│  ✗ Users impacted                                               │
│  ✗ Revenue lost                                                 │
│  ✗ Reputation damaged                                           │
│                                                                 │
│  With Chaos Engineering:                                        │
│  ✓ Find out in a controlled environment                         │
│  ✓ Fix issues before users see them                             │
│  ✓ Build confidence in system resilience                       │
│  ✓ Continuously improve                                         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Chaos Engineering Principles**

**The Four Principles of Chaos Engineering:**

```
┌─────────────────────────────────────────────────────────────────┐
│              PRINCIPLES OF CHAOS ENGINEERING                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Define Steady State                                         │
│     ─────────────────────────────────────────────────────────   │
│     • What does "normal" look like?                            │
│     • Measure: latency, error rate, throughput                  │
│     • Establish baseline metrics                                │
│                                                                 │
│  2. Hypothesize                                                 │
│     ─────────────────────────────────────────────────────────   │
│     • What do you think will happen?                           │
│     • "If we kill server X, steady state maintained"            │
│     • Make specific, testable predictions                       │
│                                                                 │
│  3. Introduce Real-World Events                                 │
│     ─────────────────────────────────────────────────────────   │
│     • Simulate failures: server crash, network partition, etc.  │
│     • Start small, increase gradually                           │
│     • Use automation for reproducibility                        │
│                                                                 │
│  4. Verify Hypothesis                                           │
│     ─────────────────────────────────────────────────────────   │
│     • Did steady state hold?                                   │
│     • If yes: confidence increased                              │
│     • If no: investigate and fix                                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **The Chaos Engineering Process**

```
┌─────────────────────────────────────────────────────────────────┐
│              CHAOS ENGINEERING PROCESS                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Start                                                           │
│   │                                                              │
│   ↓                                                              │
│  ┌─────────────────────┐                                        │
│  │  1. Define System   │                                        │
│  │     and Metrics     │                                        │
│  │                     │                                        │
│  │  - What's critical? │                                        │
│  │  - What to measure? │                                        │
│  └──────────┬──────────┘                                        │
│             │                                                    │
│             ↓                                                    │
│  ┌─────────────────────┐                                        │
│  │  2. Form Hypothesis │                                        │
│  │                     │                                        │
│  │  - What will break? │                                        │
│  │  - What will hold?  │                                        │
│  └──────────┬──────────┘                                        │
│             │                                                    │
│             ↓                                                    │
│  ┌─────────────────────┐                                        │
│  │  3. Design Experiment│                                       │
│  │                     │                                        │
│  │  - Select failure   │                                        │
│  │  - Define blast     │                                        │
│  │    radius           │                                        │
│  └──────────┬──────────┘                                        │
│             │                                                    │
│             ↓                                                    │
│  ┌─────────────────────┐      ┌─────────────────┐               │
│  │  4. Run Experiment  │────→│ Measure Impact  │               │
│  │                     │      └────────┬────────┘               │
│  │  - Inject failure   │               │                        │
│  │  - Monitor metrics  │               ↓                        │
│  └──────────┬──────────┘      ┌─────────────────┐               │
│             │                │ Compare with    │               │
│             ↓                │ Baseline        │               │
│  ┌─────────────────────┐      └────────┬────────┘               │
│  │  5. Analyze Results │               │                        │
│  │                     │               │                        │
│  │  - Did it hold?     │               ↓                        │
│  └──────────┬──────────┘      ┌─────────────────┐               │
│             │                │ Hypothesis?     │               │
│             ↓                │                 │               │
│     ┌───────────────┐       │  ┌─────────┐    │               │
│     │  Yes →       │───────│→ │  True   │    │               │
│     │  Confidence   │       │  └────┬────┘    │               │
│     │  Increased    │       │       │         │               │
│     └───────────────┘       │       ↓         │               │
│                            │  ┌─────────┐    │               │
│     ┌───────────────┐       │→ │  False  │────┘               │
│     │  No →         │       │  └────┬────┘                    │
│     │  Investigate   │       │       │                         │
│     │  and Fix       │       │       ↓                         │
│     └───────┬───────┘       │  ┌─────────┐                     │
│             │               │→ │ Improve │                     │
│             │               │  │ System  │                     │
│             └───────────────│  └─────────┘                     │
│                             │                                  │
│                             └──────────┘                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Common Chaos Experiments**

| Experiment Type | What It Tests | Example |
|-----------------|---------------|---------|
| **Pod Kill** | Instance recovery | Randomly terminate Kubernetes pods |
| **Network Latency** | Timeout handling | Add 500ms delay to API calls |
| **Network Partition** | Split-brain prevention | Isolate services from each other |
| **Resource Exhaustion** | Graceful degradation | Limit CPU/memory for a service |
| **Dependency Failure** | Circuit breakers | Take down a downstream service |
| **Disk Full** | Storage handling | Fill up disk space |
| **Clock Skew** | Time-based logic | Change system time |
| **Packet Loss** | Network resilience | Drop 10% of packets |

**Code Example — Chaos Monkey (Random Instance Kill):**

```python
import random
import time
import threading

class ChaosMonkey:
    """
    A chaos engineering tool that randomly terminates instances.
    Tests system resilience to instance failures.
    """
    
    def __init__(self, instances, kill_probability=0.1, check_interval=5):
        """
        Initialize chaos monkey.
        
        Args:
            instances: List of instances to potentially kill
            kill_probability: Chance to kill each instance per check
            check_interval: Seconds between checks
        """
        self.instances = instances
        self.kill_probability = kill_probability
        self.check_interval = check_interval
        self.running = False
        self.killed_instances = []
        self.lock = threading.Lock()
    
    def start(self):
        """Start chaos monkey."""
        print("🐒 Chaos Monkey started!")
        print(f"  Instances to terrorize: {len(self.instances)}")
        print(f"  Kill probability: {self.kill_probability:.0%}")
        print(f"  Check interval: {self.check_interval}s\n")
        
        self.running = True
        thread = threading.Thread(target=self._chaos_loop, daemon=True)
        thread.start()
    
    def stop(self):
        """Stop chaos monkey."""
        print("\n🐒 Chaos Monkey stopped!")
        self.running = False
    
    def _chaos_loop(self):
        """Main chaos loop."""
        while self.running:
            time.sleep(self.check_interval)
            
            if not self.running:
                break
            
            self._maybe_kill_instance()
    
    def _maybe_kill_instance(self):
        """Randomly kill an instance."""
        # Find alive instances
        alive_instances = [
            inst for inst in self.instances 
            if inst.is_healthy
        ]
        
        if not alive_instances:
            return
        
        # Decide whether to kill
        if random.random() < self.kill_probability:
            # Select random instance
            victim = random.choice(alive_instances)
            
            # Kill it
            with self.lock:
                victim.fail()
                self.killed_instances.append({
                    'instance_id': victim.instance_id,
                    'time': time.time()
                })
            
            print(f"🔨 Chaos Monkey killed Instance {victim.instance_id}!")
            print(f"   Remaining healthy: {len([i for i in self.instances if i.is_healthy])}/{len(self.instances)}\n")
    
    def get_stats(self):
        """Get chaos monkey statistics."""
        with self.lock:
            return {
                'total_killed': len(self.killed_instances),
                'kill_probability': self.kill_probability,
                'instances': len(self.instances),
                'healthy_instances': len([i for i in self.instances if i.is_healthy])
            }


# Demo: Chaos Monkey vs. System
print("=== Chaos Engineering Demo ===\n")

# Create a cluster of instances
instances = [ActiveInstance(i) for i in range(10)]

# Create load balancer
lb = ActiveActiveLoadBalancer(instances)

# Start chaos monkey
chaos_monkey = ChaosMonkey(
    instances=instances,
    kill_probability=0.3,  # 30% chance every 3 seconds
    check_interval=3
)
chaos_monkey.start()

# Simulate traffic
print("Simulating traffic under chaos...\n")

request_count = 0
for i in range(100):
    try:
        result = lb.route_request(f"req-{request_count}")
        if i % 10 == 0:
            print(f"Request {request_count}: OK")
    except Exception as e:
        if i % 10 == 0:
            print(f"Request {request_count}: {e}")
    
    request_count += 1
    time.sleep(0.2)

# Stop chaos monkey
chaos_monkey.stop()

# Show results
print("\n--- Final Results ---\n")
stats = chaos_monkey.get_stats()
print(f"Chaos Monkey Statistics:")
print(f"  Instances: {stats['instances']}")
print(f"  Healthy instances: {stats['healthy_instances']}")
print(f"  Total killed: {stats['total_killed']}")
print(f"  Kill probability: {stats['kill_probability']:.0%}")

print("\nService Statistics:")
for instance in instances:
    status = "✓" if instance.is_healthy else "✗"
    print(f"  {status} Instance {instance.instance_id}: {instance.requests_served} requests")

print("\n🐒 Chaos engineering teaches us to expect failures and design accordingly!")
```

**Output:**
```
=== Chaos Engineering Demo ===

🐒 Chaos Monkey started!
  Instances to terrorize: 10
  Kill probability: 30%
  Check interval: 3s

Simulating traffic under chaos...

Request 0: OK
Request 10: OK
🔨 Chaos Monkey killed Instance 3!
   Remaining healthy: 9/10

Request 20: OK
Request 30: OK
Request 40: OK
🔨 Chaos Monkey killed Instance 7!
   Remaining healthy: 8/10

Request 50: OK
🔨 Chaos Monkey killed Instance 1!
   Remaining healthy: 7/10

Request 60: OK
🔨 Chaos Monkey killed Instance 9!
   Remaining healthy: 6/10

Request 70: OK
🔨 Chaos Monkey killed Instance 2!
   Remaining healthy: 5/10

Request 80: OK
Request 90: OK

🐒 Chaos Monkey stopped!

--- Final Results ---

Chaos Monkey Statistics:
  Instances: 10
  Healthy instances: 5
  Total killed: 5
  Kill probability: 30%

Service Statistics:
  ✓ Instance 0: 12 requests
  ✗ Instance 1: 8 requests
  ✗ Instance 2: 6 requests
  ✓ Instance 4: 14 requests
  ✗ Instance 3: 7 requests
  ✓ Instance 5: 10 requests
  ✗ Instance 7: 6 requests
  ✓ Instance 6: 11 requests
  ✗ Instance 9: 5 requests
  ✓ Instance 8: 10 requests

🐒 Chaos engineering teaches us to expect failures and design accordingly!
```

### **Chaos Engineering Tools**

| Tool | Description | Use Case |
|------|-------------|----------|
| **Chaos Monkey** | Netflix's tool for random instance termination | Cloud platforms, Kubernetes |
| **Chaos Mesh** | Kubernetes-native chaos engineering platform | K8s environments |
| **Gremlin** | SaaS platform for chaos experiments | Any infrastructure |
| **Litmus Chaos** | Open-source chaos engineering for K8s | Kubernetes |
| **AWS Fault Injection Simulator** | Managed chaos experiments on AWS | AWS workloads |

---

## **Disaster Recovery: RPO and RTO**

### **The Disaster Recovery Spectrum**

```
┌─────────────────────────────────────────────────────────────────┐
│              DISASTER RECOVERY STRATEGIES                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Disaster!                                                      │
│       │                                                         │
│       ↓                                                         │
│  ┌─────────────────────────────────────────┐                   │
│  │         How fast can you recover?       │                   │
│  └─────────────────┬───────────────────────┘                   │
│                    │                                             │
│        ┌───────────┼───────────┐                               │
│        ↓           ↓           ↓                               │
│   ┌─────────┐ ┌─────────┐ ┌─────────┐                          │
│   │ Backup  │ │ Warm    │ │ Hot     │                          │
│   │ Only    │ │ Standby │ │ Standby │                          │
│   └────┬────┘ └────┬────┘ └────┬────┘                          │
│        │          │          │                                  │
│        ↓          ↓          ↓                                  │
│    Days       Hours      Minutes                              │
│    RTO         RTO        RTO                                  │
│                                                                 │
│  Trade-off: Speed ↔ Cost                                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **RPO and RTO: The Key Metrics**

#### **RPO: Recovery Point Objective**

**Definition**: The maximum acceptable amount of data loss measured in time.

**What it means:** If your RPO is 1 hour, you can afford to lose up to 1 hour of data.

**Example:**
```
RPO = 1 hour
Last backup: 10:00 AM
Disaster strikes: 10:45 AM
Data loss: 45 minutes (acceptable, < 1 hour RPO)

If disaster strikes at 11:30 AM:
Data loss: 1.5 hours (unacceptable, > 1 hour RPO)
```

**RPO by Use Case:**

| Use Case | Typical RPO | Reasoning |
|----------|-------------|-----------|
| Financial transactions | 0 seconds | Can't lose any transaction data |
| E-commerce orders | < 1 minute | Can't lose orders |
| User-generated content | 5-15 minutes | Acceptable to lose some posts/comments |
| Analytics data | 1-24 hours | Can recompute from logs |
| Static content | 1-7 days | Can be restored from backups |

#### **RTO: Recovery Time Objective**

**Definition:** The maximum acceptable length of time to restore a service after a disaster.

**What it means:** If your RTO is 4 hours, your service must be back up within 4 hours of a disaster.

**Example:**
```
RTO = 4 hours
Disaster strikes: 10:00 AM
Service restored: 1:30 PM
Downtime: 3.5 hours (acceptable, < 4 hour RTO)

If service restored at 3:00 PM:
Downtime: 5 hours (unacceptable, > 4 hour RTO)
```

**RTO by Use Case:**

| Use Case | Typical RTO | Reasoning |
|----------|-------------|-----------|
| Critical infrastructure | < 5 minutes | Must be always available |
| Payment processing | < 15 minutes | Revenue impact |
| User-facing applications | < 1 hour | User experience |
| Internal tools | 4-8 hours | Business continuity |
| Development environments | 1-2 days | Less critical |

### **RPO/RTO Trade-off Matrix**

```
┌─────────────────────────────────────────────────────────────────┐
│              RPO/RTO TRADE-OFF MATRIX                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│                        RPO (Data Loss)                          │
│                    ┌─────────────────────┐                      │
│                    │                     │                      │
│           Low      │         High        │                      │
│         (seconds)  │      (days)        │                      │
│                    │                     │                      │
│         ┌──────────┼──────────┬──────────┐                     │
│         │          │          │          │                     │
│  RTO   │   HIGH   │  MEDIUM  │   LOW    │                     │
│  (Rec. │  (mins)  │ (hours)  │ (days)   │                     │
│  Time) │          │          │          │                     │
│         │ ┌───────┐│ ┌────────┐│ ┌───────┐│                    │
│         │ │       ││ │        ││ │       ││                    │
│   Low   │ │ Sync  ││ │ Warm   ││ │ Backup││                    │
│  (mins) │ │ Rep   ││ │ Standby││ │ Only  ││                    │
│         │ │ High  ││ │ Med    ││ │ Low   ││                    │
│         │ │ Cost  ││ │ Cost   ││ │ Cost  ││                    │
│         │ └───────┘│ └────────┘│ └───────┘│                    │
│         │          │          │          │                     │
│         │ ┌───────┐│ ┌────────┐│ ┌───────┐│                    │
│         │ │       ││ │        ││ │       ││                    │
│  Med    │ │ Active││ │ Async  ││ │ Backup││                    │
│  (hours)│ │ Active││ │ Rep    ││ │ With  ││                    │
│         │ │ Highest││ │ High   ││ │ Low   ││                    │
│         │ │ Cost   ││ │ Cost   ││ │ Cost  ││                    │
│         │ └───────┘│ └────────┘│ └───────┘│                    │
│         │          │          │          │                     │
│         │ ┌───────┐│ ┌────────┐│ ┌───────┐│                    │
│         │ │       ││ │        ││ │       ││                    │
│  High   │ │ Active││ │ Backup ││ │ Backup││                    │
│  (days) │ │ Active││ │ Only   ││ │ Only  ││                    │
│         │ │ + Geo ││ │        ││ │        ││                    │
│         │ │ Cost   ││ │ Med    ││ │ Low   ││                    │
│         │ └───────┘│ └────────┘│ └───────┘│                    │
│         │          │          │          │                     │
│         └──────────┴──────────┴──────────┘                     │
│                                                                 │
│  Cost increases as RPO decreases and RTO decreases             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Designing for Your RPO/RTO**

**Step 1: Define Your Requirements**

```
Business Impact Analysis:

Questions:
1. What's the cost of downtime per hour?
2. What's the cost of data loss?
3. What are the regulatory requirements?
4. What's the impact on customer trust?
5. What SLAs have we committed to?

Example: E-commerce Platform
- Downtime cost: $10,000/hour
- Data loss cost: $100,000 for lost orders
- Regulatory: PCI DSS requires data protection
- Customer impact: High visibility
- Committed SLA: 99.9% uptime (43 minutes/month downtime)

Decided:
- RPO: 5 minutes (can't lose orders)
- RTO: 1 hour (need to restore quickly)
```

**Step 2: Design the Architecture**

```
Architecture for RPO=5min, RTO=1hour:

Components:
1. Database:
   - Primary-replica setup
   - Synchronous replication to 1 replica
   - Asynchronous replication to DR region
   - Continuous backups every 5 minutes

2. Application:
   - Stateless, auto-scaling
   - Blue-green deployment
   - Infrastructure as Code (Terraform)

3. Disaster Recovery:
   - Warm standby in DR region
   - Automated failover scripts
   - Regular DR drills (quarterly)

4. Monitoring:
   - Real-time health checks
   - Automated alerts
   - Runbook documentation
```

**Step 3: Implement and Test**

```
Implementation Checklist:

□ Data replication configured and tested
□ Backup automation in place
□ Failover procedures documented
□ DR region provisioned
□ Monitoring and alerting configured
□ Runbook created for all scenarios
□ Team trained on DR procedures
□ DR drill conducted and documented
□ RPO/RTO validated in testing

Testing:
- Simulate database failure
- Simulate region outage
- Simulate network partition
- Measure actual RPO/RTO
- Adjust if targets not met
```

### **Example Disaster Recovery Plan**

```
┌─────────────────────────────────────────────────────────────────┐
│              DISASTER RECOVERY PLAN EXAMPLE                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Service: Payment Processing API                                 │
│                                                                 │
│  RPO: 1 minute (can't lose transactions)                        │
│  RTO: 15 minutes (must restore quickly)                         │
│                                                                 │
│  ─────────────────────────────────────────────────────────────  │
│                                                                 │
│  Architecture:                                                  │
│  - Primary database: PostgreSQL in us-east-1                    │
│  - Synchronous replica: PostgreSQL in us-east-1                │
│  - Async DR replica: PostgreSQL in us-west-2                    │
│  - Application: Kubernetes, auto-scaling                        │
│  - DR region: Warm standby in us-west-2                         │
│                                                                 │
│  ─────────────────────────────────────────────────────────────  │
│                                                                 │
│  Disaster Scenarios:                                            │
│                                                                 │
│  1. Database Instance Failure                                   │
│     Detection: Automated health check (5 seconds)               │
│     Response:                                                    │
│       - Promote synchronous replica to primary                  │
│       - Update DNS to point to new primary                      │
│       - Reprovision failed instance                             │
│       - Add as new replica                                      │
│     RPO: 0 (synchronous replication)                            │
│     RTO: 2 minutes                                              │
│                                                                 │
│  2. Region Outage (us-east-1)                                   │
│     Detection: Cloud provider monitoring (1 minute)             │
│     Response:                                                    │
│       - Activate warm standby in us-west-2                      │
│       - Promote DR replica to primary                           │
│       - Update DNS to us-west-2                                 │
│       - Scale up DR application                                 │
│       - Notify team                                             │
│     RPO: <1 minute (async replication)                          │
│     RTO: 15 minutes                                             │
│                                                                 │
│  3. Data Corruption                                             │
│     Detection: Application integrity checks (continuous)        │
│     Response:                                                    │
│       - Identify point of corruption                            │
│       - Restore from latest good backup                         │
│       - Replay transaction log                                  │
│       - Validate data integrity                                 │
│     RPO: 1 minute (based on backup frequency)                   │
│     RTO: 30 minutes (restore + replay)                          │
│                                                                 │
│  ─────────────────────────────────────────────────────────────  │
│                                                                 │
│  Communication Plan:                                            │
│  - Page on-call engineer immediately                            │
│  - Notify engineering team within 5 minutes                     │
│  - Notify management within 15 minutes                          │
│  - Customer communication if outage > 30 minutes                │
│  - Post-mortem within 48 hours                                  │
│                                                                 │
│  ─────────────────────────────────────────────────────────────  │
│                                                                 │
│  Testing:                                                       │
│  - Monthly failover drills                                      │
│  - Quarterly full-region failover test                          │
│  - Annual disaster recovery simulation                          │
│  - Update plan based on test results                            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **Chapter Summary**

### **Key Takeaways**

| Concept | Summary |
|---------|---------|
| **Failure Modes** | Systems fail in three ways: crashes (stop responding), network partitions (can't communicate), and Byzantine failures (arbitrary/malicious behavior) |
| **Redundancy** | Active-passive: one active, standby takes over; Active-active: all instances serve traffic, no failover needed |
| **Retry Strategies** | Exponential backoff increases delay between retries; Jitter adds randomness to prevent thundering herd; Circuit breaker stops calling failing services |
| **Bulkheads** | Isolate resources so one failing component doesn't cascade to the entire system |
| **Chaos Engineering** | Proactively break systems to build resilience; define steady state, hypothesize, introduce failures, verify |
| **Disaster Recovery** | RPO (Recovery Point Objective) = max acceptable data loss; RTO (Recovery Time Objective) = max acceptable downtime |

### **Reliability Checklist**

Use this checklist when designing reliable systems:

```
Failure Detection:
□ Health checks implemented
□ Monitoring configured for all components
□ Alerting set up for critical failures
□ Automated failure detection

Redundancy:
□ Critical components have redundancy
□ Failover mechanisms tested
□ Data replication configured
□ No single points of failure

Retry and Circuit Breakers:
□ Retry logic with exponential backoff
□ Jitter added to prevent thundering herd
□ Circuit breakers for downstream services
□ Timeout values configured appropriately

Bulkheads:
□ Thread pools per service
□ Connection pools bounded
□ Memory limits per component
□ Queues with size limits

Chaos Engineering:
□ Chaos experiments defined
□ Regular failure injection
□ Resilience validated
□ Issues addressed proactively

Disaster Recovery:
□ RPO and RTO defined
□ Backup strategy in place
□ DR region configured
□ Failover procedures documented
□ DR drills conducted regularly
□ Team trained on procedures

Testing:
□ Failure scenarios tested
□ RPO/RTO validated
□ Runbooks created
□ Team trained
```

### **Common Mistakes to Avoid**

| Mistake | Why It's Bad | How to Fix It |
|---------|--------------|---------------|
| Not testing failure scenarios | You won't know if system works until it's too late | Implement chaos engineering and regular DR drills |
| Assuming failures won't happen | They will, at the worst time | Design for failure from the start |
| No retry logic | Transient failures cause permanent outages | Add exponential backoff with jitter |
| Infinite retries | Wastes resources, hangs system | Add max attempts and circuit breakers |
| No bulkheads | One bad service takes down entire system | Isolate resources with bulkheads |
| RPO/RTO undefined | Can't design proper DR strategy | Conduct business impact analysis |
| Only testing happy path | System fails under stress | Test failure scenarios regularly |
| Manual DR procedures | Humans make mistakes under pressure | Automate as much as possible |

---

## **Exercises**

### **Exercise 1: Design a Reliable System**

Design a highly available URL shortener with:
- RPO: 0 seconds (no data loss)
- RTO: < 5 minutes
- 99.99% uptime

**Hint:** Consider multi-region deployment, synchronous replication, automated failover.

### **Exercise 2: Implement a Circuit Breaker**

Implement a circuit breaker for a database client:
- Opens after 3 consecutive failures
- Closes after 2 consecutive successes (from half-open)
- 30-second timeout in open state

### **Exercise 3: Chaos Experiment Design**

Design a chaos experiment for a microservices architecture:
- What steady state metrics will you measure?
- What failure will you inject?
- What's your hypothesis?
- How will you verify results?

---

## **Further Reading**

| Resource | Description |
|----------|-------------|
| [Chaos Engineering](https://principlesofchaos.org/) | Official principles of chaos engineering |
| [Release It!](https://www.amazon.com/Release-Production-Ready-Michael-Nygard/dp/1680502395) | Michael Nygard's book on production-ready software |
| [Site Reliability Engineering](https://sre.google/books/) | Google's book on reliability engineering |
| [AWS Well-Architected Framework](https://aws.amazon.com/architecture/well-architected/) | AWS reliability best practices |
| [Gremlin](https://www.gremlin.com/) | Chaos engineering platform |

---

**Next:** In Chapter 12, we'll explore **Security in Distributed Systems**, covering authentication, authorization, encryption, and how to protect your systems from common vulnerabilities. We'll discuss OAuth 2.0, JWT, API security, and zero-trust architecture.

